<h1 style="text-align: center;">Creation of base tabel</h1>

## Exploring and cleaning the data
## Joining tables
## Calculate the aggregate variables based on the granularity level of customer ID
## Joining all tables and crate the base table

In [1]:
#Installing necessary packages.
#install.packages("UpSetR")
#install.packages("readxl")

In [2]:
#loading necessary packages.
library(data.table)
library(UpSetR)
library(tidyr)
library(dplyr)
library(ggplot2)
library(lubridate)
library(naniar)
library(readxl)


Warning message:
"package 'data.table' was built under R version 3.6.3"Warning message:
"package 'UpSetR' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'lubridate' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from 'package:base':

    date, intersect, setdiff, union

Warning message:
"package 'readxl' was built under R version 3.6.3"

<h2 style="text-align: center;">Demographics table</h2>

We separate the rdata file into three .csv datasets files. From now on we will use this .csv files along with the excel appendix file for the creation of base table

In [3]:
# Importing demographic.csv file previously saved as Demographics_raw
Demographics_raw<-fread("./Data/Demographics_raw.csv")

# Importing CountryName.xlxs files sheet from Appendix file
CountryName<-read_excel("./Data/Appendices Group Assignment.xlsx",sheet="Appendix 2")

# Converting CountryName as data.table type dataset

CountryName<-data.table(CountryName)
# Importing country codes file in other to use it in the map for plotly
country <- read.csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv")

# Country and Code variable selection
country<-country%>% select(COUNTRY,CODE)

# CountryName and country dataset merged
CountryName<-merge(x = CountryName, y = country, by.x = "Country Name", by.y="COUNTRY", all.x = TRUE)

# Importing  Language Description excel files sheet from Appendix
LanguageDescription<-read_excel("./Data/Appendices Group Assignment.xlsx",sheet="Appendix 3")

# Converting LanguageDescription as data.table type dataset 
LanguageDescription<-data.table(LanguageDescription)

# Importing Application Description excel files sheet from Appendix
ApplicationDescription<-read_excel("./Data/Appendices Group Assignment.xlsx",sheet="Appendix 4")

# Converting ApplicationDescription as data.table type dataset 
ApplicationDescription<-data.table(ApplicationDescription)

# Importing Product Description excel file sheet from Appendix
ProductDescription<-read_excel("./Data/Appendices Group Assignment.xlsx",sheet="Appendix 1")

# Converting ProductDescription as data.table type dataset 
ProductDescription<-data.table(ProductDescription)

In [4]:
# Cleaning and joining to create final demographic table

# There are 2 rows in FirstAct that we will drop
Demographics_raw<-Demographics_raw[FirstAct!=is.na(FirstAct),,]

# The data time is between "2005-02-01","2005-02-27" but as we want to insure we do this subset as well 
Demographics_raw<-Demographics_raw[RegDate %between% c("2005-02-01","2005-02-27"),,]

# Changing the types of columns to date
Demographics_raw$FirstPay<-ymd(Demographics_raw$FirstPay)
Demographics_raw$FirstAct<-ymd(Demographics_raw$FirstAct)
Demographics_raw$FirstSp<-ymd(Demographics_raw$FirstSp)
Demographics_raw$FirstCa<-ymd(Demographics_raw$FirstCa)
Demographics_raw$FirstGa<-ymd(Demographics_raw$FirstGa)
Demographics_raw$FirstPo<-ymd(Demographics_raw$FirstPo)

# Creation of gender type coluvariable based on the description provided with the raw datasets

# There is 1 missing value in gender and as the male proportion is more than female we choose to convert that missing to male
Demographics_raw[is.na(Gender)==TRUE,Gender_type:="Male",]
Demographics_raw[Gender==1,Gender_type:="Male",]
Demographics_raw[Gender==0,Gender_type:="Female",]

# Creation of demographic dataset by merging it to country, language and application description

# Merging with country description
Demographics<-merge(x = Demographics_raw, y = CountryName, by.x = "Country", by.y="Country", all.x = TRUE)

# Merging with Language description
Demographics<-merge(x = Demographics, y = LanguageDescription, by.x = "Language", by.y="Language", all.x = TRUE)

# Merging with Application description
Demographics<-merge(x = Demographics, y = ApplicationDescription, by.x = "ApplicationID", by.y="ApplicationID", all.x = TRUE)

# Dropping the unnecessary variables of the raw dataset
Demographics<-Demographics[,`:=`(Country=NULL,Language=NULL,ApplicationID=NULL,Gender=NULL),]

# Rename variables
Demographics<-Demographics%>%rename(Gender=Gender_type,Country="Country Name",Language="Language Description",Application="Application Description")
# Checking the final Demographics dataset variables
head(Demographics)

Warning message:
" 1234 failed to parse."Warning message:
" 36759 failed to parse."Warning message:
" 39374 failed to parse."Warning message:
" 40505 failed to parse."

UserID,RegDate,FirstPay,FirstAct,FirstSp,FirstCa,FirstGa,FirstPo,Gender,Country,CODE,Language,Application
1324448,2005-02-01,2005-02-02,2005-02-02,2005-02-02,NA,NA,NA,Female,Australia,AUS,English,BETANDWIN.COM
1326405,2005-02-02,2005-02-22,2005-02-22,2005-02-22,NA,NA,NA,Male,Australia,AUS,English,BETANDWIN.COM
1334041,2005-02-05,2005-02-09,2005-06-05,2005-06-05,NA,NA,NA,Male,Australia,AUS,English,BETANDWIN.COM
1342238,2005-02-08,2005-02-08,2005-02-08,2005-02-08,2005-02-10,NA,NA,Male,Australia,AUS,English,BETANDWIN.COM
1342624,2005-02-08,2005-02-08,2005-02-09,2005-02-09,NA,NA,NA,Male,Australia,AUS,English,BETANDWIN.COM
1349161,2005-02-11,2005-02-16,2005-02-17,2005-02-17,2005-03-28,NA,NA,Male,Australia,AUS,English,BETANDWIN.COM


In [5]:
# Importing pokerchipconversion.csv file previously saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
head(PokerChipConversions_raw)

UserID,TransDateTime,TransType,TransAmount
1324355,2005-06-12 00:37:00,124,8.9999
1324355,2005-06-12 00:51:00,124,1.9999
1324355,2005-06-12 01:14:00,124,4.9999
1324355,2005-06-12 02:01:00,24,1.8069
1324355,2005-06-14 23:35:00,124,4.9999
1324355,2005-06-14 23:45:00,124,4.9999


<h2 style="text-align: center;">Transaction table</h2>

In [6]:
# Importing pokerchipconversion.csv file previously saved
trans_raw<-fread("./Data/PokerChipConversions_raw.csv")

# Subsetting  the PokerChipConversions to Sell dataset based on transaction type = 24

# Aggregating and creating variables for payout dataset

# Frequency
trans_agg<-trans_raw[,"Total Transaction Frequency":=.N,by=UserID]

# Total Amount
trans_agg<-trans_raw[,"Total Transaction Amount":=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
trans_agg<-unique(trans_agg,by="UserID")

# Dropping the unecessary variables of the raw dataset
trans_agg<-trans_agg[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
head(trans_agg)

UserID,Total Transaction Frequency,Total Transaction Amount
1324355,8,39.088800
1324368,99,18782.472900
1324369,2,0.589891
1324371,10,188.000064
1324372,238,1779.197660
1324377,7,40.547054


<h3 style="text-align: center;">Transaction table agregate and calculation agregation metrics based on each month</h3>

In [7]:
# Importing pokerchipconversion.csv file previously saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")

# Subseting the PokerChipConversions to Sell dataset based on transaction type = 24
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# Aggregating and creating variables for payout dataset

# Frequency
PokerChipConversions_Sell_agg<-PokerChipConversions_raw_Sell[,Sell_Freq_Total:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg<-PokerChipConversions_raw_Sell[,Sell_amount_Total:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg<-unique(PokerChipConversions_Sell_agg,by="UserID")

# Dropping the unecessary columns of the raw table
PokerChipConversions_Sell_agg<-PokerChipConversions_Sell_agg[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

# Subsetting the PokerChipConversions to Sell dataset based on transaction type = 24

# Importing pokerchipconversion.csv file previously saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# Feb
PokerChipConversions_raw_Sell_Feb<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-02-01","2005-02-28")]

# Frequency
PokerChipConversions_Sell_agg_Feb<-PokerChipConversions_raw_Sell_Feb[,Sell_Freq_Feb:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_Feb<-PokerChipConversions_raw_Sell_Feb[,Sell_amount_Feb:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_Feb<-unique(PokerChipConversions_Sell_agg_Feb,by="UserID")

# Dropping the unnecessary variables of the raw table
PokerChipConversions_Sell_agg_Feb<-PokerChipConversions_Sell_agg_Feb[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]


# Importing pokerchipconversion.csv file previously saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# March
PokerChipConversions_raw_Sell_Mar<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-03-01","2005-03-31")]

# Frequency
PokerChipConversions_Sell_agg_Mar<-PokerChipConversions_raw_Sell_Mar[,Sell_Freq_Mar:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_Mar<-PokerChipConversions_raw_Sell_Mar[,Sell_amount_Mar:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_Mar<-unique(PokerChipConversions_Sell_agg_Mar,by="UserID")

# Dropping the unnecessary variables of the raw table
PokerChipConversions_Sell_agg_Mar<-PokerChipConversions_Sell_agg_Mar[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

#------------------------
#Importing pokerchipconversion.csv file previously saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# April
PokerChipConversions_raw_Sell_Apr<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-04-01","2005-04-30")]

# Frequency
PokerChipConversions_Sell_agg_Apr<-PokerChipConversions_raw_Sell_Apr[,Sell_Freq_Apr:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_Apr<-PokerChipConversions_raw_Sell_Apr[,Sell_amount_Apr:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_Apr<-unique(PokerChipConversions_Sell_agg_Apr,by="UserID")

# Dropping the unnecessary variables of the raw table
PokerChipConversions_Sell_agg_Apr<-PokerChipConversions_Sell_agg_Apr[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

#------------------------
# Importing pokerchipconversion.csv file previously saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# May
PokerChipConversions_raw_Sell_May<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-05-01","2005-05-31")]

# Frequency
PokerChipConversions_Sell_agg_May<-PokerChipConversions_raw_Sell_May[,Sell_Freq_May:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_May<-PokerChipConversions_raw_Sell_May[,Sell_amount_May:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_May<-unique(PokerChipConversions_Sell_agg_May,by="UserID")

#Dropping the unnecessary variables of the raw table
PokerChipConversions_Sell_agg_May<-PokerChipConversions_Sell_agg_May[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

#------------------------
# Importing pokerchipconversion.csv file previously saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# June
PokerChipConversions_raw_Sell_Jun<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-06-01","2005-06-30")]

# Frequency
PokerChipConversions_Sell_agg_Jun<-PokerChipConversions_raw_Sell_Jun[,Sell_Freq_Jun:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_Jun<-PokerChipConversions_raw_Sell_Jun[,Sell_amount_Jun:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_Jun<-unique(PokerChipConversions_Sell_agg_Jun,by="UserID")

# Droping the unecessary columns of the raw table
PokerChipConversions_Sell_agg_Jun<-PokerChipConversions_Sell_agg_Jun[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

#------------------------
# Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# July
PokerChipConversions_raw_Sell_Jul<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-07-01","2005-07-31")]

# Frequency
PokerChipConversions_Sell_agg_Jul<-PokerChipConversions_raw_Sell_Jul[,Sell_Freq_Jul:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_Jul<-PokerChipConversions_raw_Sell_Jul[,Sell_amount_Jul:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_Jul<-unique(PokerChipConversions_Sell_agg_Jul,by="UserID")

# Dropping the unecessary columns of the raw table
PokerChipConversions_Sell_agg_Jul<-PokerChipConversions_Sell_agg_Jul[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

#------------------------
# Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# August
PokerChipConversions_raw_Sell_Aug<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-08-01","2005-08-31")]

# Frequency
PokerChipConversions_Sell_agg_Aug<-PokerChipConversions_raw_Sell_Aug[,Sell_Freq_Aug:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_Aug<-PokerChipConversions_raw_Sell_Aug[,Sell_amount_Aug:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_Aug<-unique(PokerChipConversions_Sell_agg_Aug,by="UserID")

# Dropping the unecessary columns of the raw table
PokerChipConversions_Sell_agg_Aug<-PokerChipConversions_Sell_agg_Aug[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

#------------------------
# Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Sell<-PokerChipConversions_raw[TransType==24,,]

# September
PokerChipConversions_raw_Sell_Sep<-PokerChipConversions_raw_Sell[TransDateTime %between% c("2005-09-01","2005-09-30")]

# Frequency
PokerChipConversions_Sell_agg_Sep<-PokerChipConversions_raw_Sell_Sep[,Sell_Freq_Sep:=.N,by=UserID]

# Total Amount
PokerChipConversions_Sell_agg_Sep<-PokerChipConversions_raw_Sell_Sep[,Sell_amount_Sep:=sum(TransAmount),by=UserID]

# Dropping the duplicate Rows
PokerChipConversions_Sell_agg_Sep<-unique(PokerChipConversions_Sell_agg_Sep,by="UserID")

# Dropping the unecessary columns of the raw table
PokerChipConversions_Sell_agg_Sep<-PokerChipConversions_Sell_agg_Sep[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_Feb, by = "UserID", all.x = TRUE)
PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_Mar, by = "UserID", all.x = TRUE)
PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_Apr, by = "UserID", all.x = TRUE)
PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_May, by = "UserID", all.x = TRUE)
PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_Jun, by = "UserID", all.x = TRUE)
PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_Jul, by = "UserID", all.x = TRUE)
PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_Aug, by = "UserID", all.x = TRUE)
PokerChipConversions_Sell_agg<-merge(x = PokerChipConversions_Sell_agg, y = PokerChipConversions_Sell_agg_Sep, by = "UserID", all.x = TRUE)


head(PokerChipConversions_Sell_agg)

UserID,Sell_Freq_Total,Sell_amount_Total,Sell_Freq_Feb,Sell_amount_Feb,Sell_Freq_Mar,Sell_amount_Mar,Sell_Freq_Apr,Sell_amount_Apr,Sell_Freq_May,Sell_amount_May,Sell_Freq_Jun,Sell_amount_Jun,Sell_Freq_Jul,Sell_amount_Jul,Sell_Freq_Aug,Sell_amount_Aug,Sell_Freq_Sep,Sell_amount_Sep
1324355,2,8.2593000,NA,NA,NA,NA,NA,NA,NA,NA,2,8.2593,NA,NA,NA,NA,NA,NA
1324368,32,9620.8147000,NA,NA,NA,NA,20,5035.346,12,4585.469,NA,NA,NA,NA,NA,NA,NA,NA
1324369,1,0.2949455,NA,NA,1,0.2949455,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324371,5,65.3674456,5,65.36745,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324372,116,918.4136762,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,103,813.1091
1324377,2,12.9949062,2,12.99491,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [8]:
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
#Subset the PokerChipConversions to Buy table based on transaction type = 124
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#aggregate and crate variables for payout table
#Frequency
PokerChipConversions_Buy_agg<-PokerChipConversions_raw_Buy[,Buy_Freq_Total:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg<-PokerChipConversions_raw_Buy[,Buy_amount_Total:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg<-unique(PokerChipConversions_Buy_agg,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg<-PokerChipConversions_Buy_agg[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

#Subset the PokerChipConversions to Buy table based on transaction type = 124
#------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#feb
PokerChipConversions_raw_Buy_Feb<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-02-01","2005-02-28")]
#Frequency
PokerChipConversions_Buy_agg_Feb<-PokerChipConversions_raw_Buy_Feb[,Buy_Freq_Feb:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_Feb<-PokerChipConversions_raw_Buy_Feb[,Buy_amount_Feb:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_Feb<-unique(PokerChipConversions_Buy_agg_Feb,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_Feb<-PokerChipConversions_Buy_agg_Feb[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
# #------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#March
PokerChipConversions_raw_Buy_Mar<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-03-01","2005-03-31")]
#Frequency
PokerChipConversions_Buy_agg_Mar<-PokerChipConversions_raw_Buy_Mar[,Buy_Freq_Mar:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_Mar<-PokerChipConversions_raw_Buy_Mar[,Buy_amount_Mar:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_Mar<-unique(PokerChipConversions_Buy_agg_Mar,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_Mar<-PokerChipConversions_Buy_agg_Mar[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
#------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#April
PokerChipConversions_raw_Buy_Apr<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-04-01","2005-04-30")]
#Frequency
PokerChipConversions_Buy_agg_Apr<-PokerChipConversions_raw_Buy_Apr[,Buy_Freq_Apr:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_Apr<-PokerChipConversions_raw_Buy_Apr[,Buy_amount_Apr:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_Apr<-unique(PokerChipConversions_Buy_agg_Apr,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_Apr<-PokerChipConversions_Buy_agg_Apr[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
#------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#May
PokerChipConversions_raw_Buy_May<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-05-01","2005-05-31")]
#Frequency
PokerChipConversions_Buy_agg_May<-PokerChipConversions_raw_Buy_May[,Buy_Freq_May:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_May<-PokerChipConversions_raw_Buy_May[,Buy_amount_May:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_May<-unique(PokerChipConversions_Buy_agg_May,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_May<-PokerChipConversions_Buy_agg_May[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
#------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#June
PokerChipConversions_raw_Buy_Jun<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-06-01","2005-06-30")]
#Frequency
PokerChipConversions_Buy_agg_Jun<-PokerChipConversions_raw_Buy_Jun[,Buy_Freq_Jun:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_Jun<-PokerChipConversions_raw_Buy_Jun[,Buy_amount_Jun:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_Jun<-unique(PokerChipConversions_Buy_agg_Jun,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_Jun<-PokerChipConversions_Buy_agg_Jun[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
#------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#July
PokerChipConversions_raw_Buy_Jul<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-07-01","2005-07-31")]
#Frequency
PokerChipConversions_Buy_agg_Jul<-PokerChipConversions_raw_Buy_Jul[,Buy_Freq_Jul:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_Jul<-PokerChipConversions_raw_Buy_Jul[,Buy_amount_Jul:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_Jul<-unique(PokerChipConversions_Buy_agg_Jul,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_Jul<-PokerChipConversions_Buy_agg_Jul[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
#------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#August
PokerChipConversions_raw_Buy_Aug<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-08-01","2005-08-31")]
#Frequency
PokerChipConversions_Buy_agg_Aug<-PokerChipConversions_raw_Buy_Aug[,Buy_Freq_Aug:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_Aug<-PokerChipConversions_raw_Buy_Aug[,Buy_amount_Aug:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_Aug<-unique(PokerChipConversions_Buy_agg_Aug,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_Aug<-PokerChipConversions_Buy_agg_Aug[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]
#------------------------
#Importing poker chip conversion csv that we saved
PokerChipConversions_raw<-fread("./Data/PokerChipConversions_raw.csv")
PokerChipConversions_raw_Buy<-PokerChipConversions_raw[TransType==124,,]
#September
PokerChipConversions_raw_Buy_Sep<-PokerChipConversions_raw_Buy[TransDateTime %between% c("2005-09-01","2005-09-30")]
#Frequency
PokerChipConversions_Buy_agg_Sep<-PokerChipConversions_raw_Buy_Sep[,Buy_Freq_Sep:=.N,by=UserID]
#Total Amount
PokerChipConversions_Buy_agg_Sep<-PokerChipConversions_raw_Buy_Sep[,Buy_amount_Sep:=sum(TransAmount),by=UserID]
#Droping the duplicate Rows
PokerChipConversions_Buy_agg_Sep<-unique(PokerChipConversions_Buy_agg_Sep,by="UserID")
#Droping the unecessary columns of the raw table
PokerChipConversions_Buy_agg_Sep<-PokerChipConversions_Buy_agg_Sep[,`:=`(TransDateTime=NULL,TransType=NULL,TransAmount=NULL),]

PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_Feb, by = "UserID", all.x = TRUE)
PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_Mar, by = "UserID", all.x = TRUE)
PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_Apr, by = "UserID", all.x = TRUE)
PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_May, by = "UserID", all.x = TRUE)
PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_Jun, by = "UserID", all.x = TRUE)
PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_Jul, by = "UserID", all.x = TRUE)
PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_Aug, by = "UserID", all.x = TRUE)
PokerChipConversions_Buy_agg<-merge(x = PokerChipConversions_Buy_agg, y = PokerChipConversions_Buy_agg_Sep, by = "UserID", all.x = TRUE)


head(PokerChipConversions_Buy_agg)

UserID,Buy_Freq_Total,Buy_amount_Total,Buy_Freq_Feb,Buy_amount_Feb,Buy_Freq_Mar,Buy_amount_Mar,Buy_Freq_Apr,Buy_amount_Apr,Buy_Freq_May,Buy_amount_May,Buy_Freq_Jun,Buy_amount_Jun,Buy_Freq_Jul,Buy_amount_Jul,Buy_Freq_Aug,Buy_amount_Aug,Buy_Freq_Sep,Buy_amount_Sep
1324355,6,30.8295000,NA,NA,NA,NA,NA,NA,NA,NA,6,30.8295,NA,NA,NA,NA,NA,NA
1324368,67,9161.6582000,NA,NA,NA,NA,34,4918.158,33,4243.5,NA,NA,NA,NA,NA,NA,NA,NA
1324369,1,0.2949455,NA,NA,1,0.2949455,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324371,5,122.6326185,5,122.63262,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324372,122,860.7839833,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,109,753.0522
1324377,5,27.5521477,5,27.55215,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


<h2 style="text-align: center;">Games Table</h2>

In [9]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)
#checking the products groups
UserDailyAggregation[,mean(ProductID),by="Product Description"]
#head of table
head(UserDailyAggregation)

Product Description,V1
Sports book fixed-odd,1
Sports book live-action,2
Casino BossMedia,4
Supertoto,5
Games VS,6
Games bwin,7
Casino Chartwell,8


ProductID,UserID,Date,Stakes,Winnings,Bets,Product Description
1,1324354,2005-02-24,20,0,2,Sports book fixed-odd
1,1324354,2005-02-25,0,0,0,Sports book fixed-odd
1,1324354,2005-02-27,20,0,2,Sports book fixed-odd
1,1324354,2005-03-03,10,0,1,Sports book fixed-odd
1,1324354,2005-03-04,10,0,1,Sports book fixed-odd
1,1324354,2005-03-05,10,0,1,Sports book fixed-odd


<h3 style="text-align: center;">Games table agregate and calculation of Loyalty and Engagement</h3>

In [10]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
games_raw<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#First Day of Play
games_agg<-games_raw[,all_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
games_agg<-games_raw[,all_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
games_agg<-games_raw[,"Total Duration":=difftime(all_Last_Date,
                                                            all_Fisrt_Date,units="days"),by=UserID]
#Duration between First and Last
games_agg<-games_raw[,"Recennt_Play":=as.integer(difftime("2005-09-30",all_Last_Date,
                                                            units="days")),by=UserID]
#One Day Retention Calculation
games_agg<-games_raw[,all_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
games_agg<-games_raw[,
                            "Total Retantion":=.(ifelse(all_Second_Date==all_Fisrt_Date+days(1),TRUE,FALSE)),]
#If customer play the game in last week or not
games_agg<-games_raw[,
                            "Recency":=.(ifelse(all_Last_Date>="2005-09-23",TRUE,FALSE)),]
#Frequency of Play
games_agg<-games_raw[,"Total Game Frequency":=.N,by=UserID]
#total Stakes
games_agg<-games_raw[,"Total Game Stakes":=sum(Stakes),by=UserID]
#Total Winning
games_agg<-games_raw[,"Total Game Winnings":=sum(Winnings),by=UserID]
#Total Bets
games_agg<-games_raw[,"Total Game Bets":=sum(Bets),by=UserID]
#Droping the duplicate User Id 
games_agg<-unique(games_agg,by="UserID")
#Droping the unnecessary columns
games_agg<-games_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,all_Second_Date=NULL,
                                all_Fisrt_Date=NULL),]

head(games_agg)


UserID,all_Last_Date,Total Duration,Recennt_Play,Total Retantion,Recency,Total Game Frequency,Total Game Stakes,Total Game Winnings,Total Game Bets
1324354,2005-09-26,214 days,3,TRUE,TRUE,136,11976.6100,11736.6100,279
1324355,2005-02-07,6 days,234,TRUE,FALSE,106,425.5600,464.5000,252
1324356,2005-09-11,221 days,18,TRUE,FALSE,75,1365.2600,910.6600,214
1324358,2005-05-03,91 days,149,TRUE,FALSE,9,336.2898,209.8575,11
1324360,2005-02-03,1 days,238,TRUE,FALSE,32,65.7427,43.1573,47
1324362,2005-09-17,218 days,12,TRUE,FALSE,7,22.0000,0.0000,7


In [11]:
games_agg<-data.frame(games_agg)

In [12]:
games_agg<-games_agg%>%mutate(Recency_Groups= case_when(
    between(Recennt_Play,0,21)~4,
    between(Recennt_Play,22,144)~3,
    between(Recennt_Play,145,207)~2,
    TRUE~1
))
games_agg<-games_agg%>%mutate(Game_freq_Groups= case_when(
    between(Total.Game.Frequency,0,8)~1,
    between(Total.Game.Frequency,9,22)~2,
    between(Total.Game.Frequency,23,52)~3,
    TRUE~4))
games_agg<-games_agg%>%mutate(game_stak_groups= case_when(
    between(Total.Game.Stakes,0,77)~1,
    between(Total.Game.Stakes,78,233)~2,
    between(Total.Game.Stakes,234,819)~3,
    TRUE~4
))
games_agg<-games_agg%>%mutate(game_wining_groups= case_when(
    between(Total.Game.Winnings,0,37)~1,
    between(Total.Game.Winnings,38,175)~2,
    between(Total.Game.Winnings,176,710)~3,
    TRUE~4
))
games_agg<-games_agg%>%mutate(game_Bets_groups= case_when(
    between(Total.Game.Bets,0,16)~1,
    between(Total.Game.Bets,17,61)~2,
    between(Total.Game.Bets,62,200)~3,
    TRUE~4
))
games_agg<-games_agg%>%mutate(RFM_Score= Recency_Groups*Game_freq_Groups*game_stak_groups)
games_agg<-games_agg%>%mutate(Loyalty_level= case_when(
    between(RFM_Score,0,4)~"Basic",
    between(RFM_Score,5,12)~"Clasic",
    between(RFM_Score,13,32)~"Premium",
    TRUE~"Gold"
))
games_agg<-games_agg%>%mutate(Engagement_Score= game_wining_groups*Game_freq_Groups*game_stak_groups*game_Bets_groups)
games_agg<-games_agg%>%mutate(Engagement_level= case_when(
    between(Engagement_Score,0,4)~"Low",
    between(Engagement_Score,5,36)~"Medium",
    between(Engagement_Score,37,128)~"High",
    TRUE~"Top"
))
games_agg<-games_agg%>%mutate(Recency_Groups= case_when(
    between(Recennt_Play,0,21)~"Top",
    between(Recennt_Play,22,144)~"High",
    between(Recennt_Play,145,207)~"Medium",
    TRUE~"Low"
))
games_agg<-games_agg%>%mutate(Game_freq_Groups= case_when(
    between(Total.Game.Frequency,0,8)~"Low",
    between(Total.Game.Frequency,9,22)~"Medium",
    between(Total.Game.Frequency,23,52)~"High",
    TRUE~"Top"
))
games_agg<-games_agg%>%mutate(game_stak_groups= case_when(
    between(Total.Game.Stakes,0,77)~"Low",
    between(Total.Game.Stakes,78,233)~"Medium",
    between(Total.Game.Stakes,234,819)~"High",
    TRUE~"Top"
))
games_agg<-games_agg%>%mutate(game_wining_groups= case_when(
    between(Total.Game.Winnings,0,37)~"Low",
    between(Total.Game.Winnings,38,175)~"Medium",
    between(Total.Game.Winnings,176,710)~"High",
    TRUE~"Top"
))
games_agg<-games_agg%>%mutate(game_Bets_groups= case_when(
    between(Total.Game.Bets,0,16)~"Low",
    between(Total.Game.Bets,17,61)~"Medium",
    between(Total.Game.Bets,62,200)~"High",
    TRUE~"Top"
))

head(games_agg)

UserID,all_Last_Date,Total.Duration,Recennt_Play,Total.Retantion,Recency,Total.Game.Frequency,Total.Game.Stakes,Total.Game.Winnings,Total.Game.Bets,Recency_Groups,Game_freq_Groups,game_stak_groups,game_wining_groups,game_Bets_groups,RFM_Score,Loyalty_level,Engagement_Score,Engagement_level
1324354,2005-09-26,214 days,3,TRUE,TRUE,136,11976.6100,11736.6100,279,Top,Top,Top,Top,Top,64,Gold,256,Top
1324355,2005-02-07,6 days,234,TRUE,FALSE,106,425.5600,464.5000,252,Low,Top,High,High,Top,12,Clasic,144,Top
1324356,2005-09-11,221 days,18,TRUE,FALSE,75,1365.2600,910.6600,214,Top,Top,Top,Top,Top,64,Gold,256,Top
1324358,2005-05-03,91 days,149,TRUE,FALSE,9,336.2898,209.8575,11,Medium,Medium,High,High,Low,12,Clasic,18,Medium
1324360,2005-02-03,1 days,238,TRUE,FALSE,32,65.7427,43.1573,47,Low,High,Low,Medium,Medium,3,Basic,12,Medium
1324362,2005-09-17,218 days,12,TRUE,FALSE,7,22.0000,0.0000,7,Top,Low,Low,Low,Low,4,Basic,1,Low


<h3 style="text-align: center;">Games table agregate and calculation agregation metrics based on each month</h3>


In [13]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#First Day of Play
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Dur:=difftime(Fix_Odd_Last_Date,
                                                            Fix_Odd_Fisrt_Date,units="days"),by=UserID]
#One Day Retention Calculation
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,
                            Fix_Odd_1Day_Retention:=.(ifelse(Fix_Odd_Second_Date==Fix_Odd_Fisrt_Date+days(1),"Returned","Not_Rerturned")),]
#If customer play the game in last week or not
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,
                            Fix_Odd_Recent:=.(ifelse(Fix_Odd_Last_Date>="2005-09-23","Recent","Not_Recent")),]
#Frequency of Play
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Freq_Total:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Stakes_Total:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Winnings_Total:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_raw_SBFO[,Fix_Odd_Bets_Total:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg<-unique(UserDailyAggregation_SBFO_agg,by="UserID")
#Droping the unnecessary columns
UserDailyAggregation_SBFO_agg<-UserDailyAggregation_SBFO_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,Fix_Odd_Second_Date=NULL,
                                Fix_Odd_Fisrt_Date=NULL,Fix_Odd_Last_Date=NULL),]

head(UserDailyAggregation_SBFO_agg)

UserID,Fix_Odd_Dur,Fix_Odd_1Day_Retention,Fix_Odd_Recent,Fix_Odd_Freq_Total,Fix_Odd_Stakes_Total,Fix_Odd_Winnings_Total,Fix_Odd_Bets_Total
1324354,218 days,Returned,Recent,117,10137.3300,10224.1200,236
1324355,240 days,Returned,Recent,99,400.8600,453.3000,231
1324356,222 days,Returned,Not_Recent,51,686.1900,285.5100,98
1324358,94 days,Returned,Not_Recent,8,247.6971,153.8756,7
1324360,235 days,Returned,Recent,29,59.9993,39.9564,40
1324362,218 days,Returned,Not_Recent,7,22.0000,0.0000,7


In [14]:
#Importing user daily csv that we saved
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#February
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#Calculatinf Frequency and total for February
UserDailyAggregation_raw_SBFO_Feb<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-02-01","2005-02-28")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_Feb<-UserDailyAggregation_raw_SBFO_Feb[,Fix_Odd_Freq_FEB:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_Feb<-UserDailyAggregation_raw_SBFO_Feb[,Fix_Odd_Stakes_FEB:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_Feb<-UserDailyAggregation_raw_SBFO_Feb[,Fix_Odd_Winnings_FEB:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_Feb<-UserDailyAggregation_raw_SBFO_Feb[,Fix_Odd_Bets_FEB:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_Feb<-unique(UserDailyAggregation_SBFO_agg_Feb,by="UserID")

UserDailyAggregation_SBFO_agg_Feb<-UserDailyAggregation_SBFO_agg_Feb[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_Feb)

UserID,Fix_Odd_Freq_FEB,Fix_Odd_Stakes_FEB,Fix_Odd_Winnings_FEB,Fix_Odd_Bets_FEB
1324354,3,40.0000,0.0000,4
1324355,21,56.3600,62.8800,57
1324356,20,109.9600,77.6300,36
1324358,3,120.7323,90.0193,4
1324360,13,16.6860,14.1960,17
1324362,3,10.0000,0.0000,2


In [15]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#MARCH
#------------------------------------
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#March
UserDailyAggregation_raw_SBFO_Mar<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-03-01","2005-03-31")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_Mar<-UserDailyAggregation_raw_SBFO_Mar[,Fix_Odd_Freq_Mar:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_Mar<-UserDailyAggregation_raw_SBFO_Mar[,Fix_Odd_Stakes_Mar:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_Mar<-UserDailyAggregation_raw_SBFO_Mar[,Fix_Odd_Winnings_Mar:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_Mar<-UserDailyAggregation_raw_SBFO_Mar[,Fix_Odd_Bets_Mar:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_Mar<-unique(UserDailyAggregation_SBFO_agg_Mar,by="UserID")
UserDailyAggregation_SBFO_agg_Mar<-UserDailyAggregation_SBFO_agg_Mar[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_Mar)

UserID,Fix_Odd_Freq_Mar,Fix_Odd_Stakes_Mar,Fix_Odd_Winnings_Mar,Fix_Odd_Bets_Mar
1324354,21,1814.1400,2000.650,31
1324355,18,66.9400,242.840,53
1324356,11,106.8900,16.160,15
1324360,5,5.7226,2.646,6
1324364,1,0.0000,3.000,0
1324368,14,269.9800,209.080,34


In [16]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#April
#------------------------------------
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#March
UserDailyAggregation_raw_SBFO_Apr<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-04-01","2005-04-30")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_Apr<-UserDailyAggregation_raw_SBFO_Apr[,Fix_Odd_Freq_Apr:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_Apr<-UserDailyAggregation_raw_SBFO_Apr[,Fix_Odd_Stakes_Apr:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_Apr<-UserDailyAggregation_raw_SBFO_Apr[,Fix_Odd_Winnings_Apr:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_Apr<-UserDailyAggregation_raw_SBFO_Apr[,Fix_Odd_Bets_Apr:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_Apr<-unique(UserDailyAggregation_SBFO_agg_Apr,by="UserID")
UserDailyAggregation_SBFO_agg_Apr<-UserDailyAggregation_SBFO_agg_Apr[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_Apr)

UserID,Fix_Odd_Freq_Apr,Fix_Odd_Stakes_Apr,Fix_Odd_Winnings_Apr,Fix_Odd_Bets_Apr
1324354,30,3781.6300,3589.62,67
1324355,16,52.9200,0.00,36
1324356,2,0.0000,0.00,0
1324360,1,0.0000,0.00,0
1324368,23,952.5200,599.67,46
1324369,2,0.8014,0.00,1


In [17]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#May
#------------------------------------
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#March
UserDailyAggregation_raw_SBFO_May<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-05-01","2005-05-31")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_May<-UserDailyAggregation_raw_SBFO_May[,Fix_Odd_Freq_May:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_May<-UserDailyAggregation_raw_SBFO_May[,Fix_Odd_Stakes_May:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_May<-UserDailyAggregation_raw_SBFO_May[,Fix_Odd_Winnings_May:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_May<-UserDailyAggregation_raw_SBFO_May[,Fix_Odd_Bets_May:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_May<-unique(UserDailyAggregation_SBFO_agg_May,by="UserID")
UserDailyAggregation_SBFO_agg_May<-UserDailyAggregation_SBFO_agg_May[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_May)

UserID,Fix_Odd_Freq_May,Fix_Odd_Stakes_May,Fix_Odd_Winnings_May,Fix_Odd_Bets_May
1324354,22,2172.6300,2144.5300,42
1324355,8,17.1500,13.6500,16
1324358,5,126.9648,63.8563,3
1324368,25,727.9400,1100.5300,58
1324369,4,31.2437,11.9847,13
1324378,2,50.0000,0.0000,4


In [18]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#June
#------------------------------------
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#March
UserDailyAggregation_raw_SBFO_Jun<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-06-01","2005-06-30")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_Jun<-UserDailyAggregation_raw_SBFO_Jun[,Fix_Odd_Freq_Jun:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_Jun<-UserDailyAggregation_raw_SBFO_Jun[,Fix_Odd_Stakes_Jun:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_Jun<-UserDailyAggregation_raw_SBFO_Jun[,Fix_Odd_Winnings_Jun:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_Jun<-UserDailyAggregation_raw_SBFO_Jun[,Fix_Odd_Bets_Jun:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_Jun<-unique(UserDailyAggregation_SBFO_agg_Jun,by="UserID")
UserDailyAggregation_SBFO_agg_Jun<-UserDailyAggregation_SBFO_agg_Jun[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_Jun)

UserID,Fix_Odd_Freq_Jun,Fix_Odd_Stakes_Jun,Fix_Odd_Winnings_Jun,Fix_Odd_Bets_Jun
1324354,4,80.0000,0.0000,7
1324355,13,51.9600,34.0300,12
1324379,22,424.0000,361.1500,52
1324383,1,631.5524,611.8164,2
1324386,10,110.0458,131.3259,24
1324403,2,123.3500,209.7000,1


In [19]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#July
#------------------------------------
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#March
UserDailyAggregation_raw_SBFO_Jul<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-07-01","2005-07-31")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_Jul<-UserDailyAggregation_raw_SBFO_Jul[,Fix_Odd_Freq_Jul:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_Jul<-UserDailyAggregation_raw_SBFO_Jul[,Fix_Odd_Stakes_Jul:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_Jul<-UserDailyAggregation_raw_SBFO_Jul[,Fix_Odd_Winnings_Jul:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_Jul<-UserDailyAggregation_raw_SBFO_Jul[,Fix_Odd_Bets_Jul:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_Jul<-unique(UserDailyAggregation_SBFO_agg_Jul,by="UserID")
UserDailyAggregation_SBFO_agg_Jul<-UserDailyAggregation_SBFO_agg_Jul[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_Jul)

UserID,Fix_Odd_Freq_Jul,Fix_Odd_Stakes_Jul,Fix_Odd_Winnings_Jul,Fix_Odd_Bets_Jul
1324354,1,0.0000,0.0000,0
1324368,4,28.3700,0.0000,3
1324379,25,389.0000,295.9700,72
1324386,8,237.6571,231.9286,9
1324404,31,3119.2400,3067.9800,330
1324405,2,0.9000,0.0000,1


In [20]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#August
#------------------------------------
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#March
UserDailyAggregation_raw_SBFO_Aug<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-08-01","2005-08-31")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_Aug<-UserDailyAggregation_raw_SBFO_Aug[,Fix_Odd_Freq_Aug:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_Aug<-UserDailyAggregation_raw_SBFO_Aug[,Fix_Odd_Stakes_Aug:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_Aug<-UserDailyAggregation_raw_SBFO_Aug[,Fix_Odd_Winnings_Aug:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_Aug<-UserDailyAggregation_raw_SBFO_Aug[,Fix_Odd_Bets_Aug:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_Aug<-unique(UserDailyAggregation_SBFO_agg_Aug,by="UserID")
UserDailyAggregation_SBFO_agg_Aug<-UserDailyAggregation_SBFO_agg_Aug[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_Aug)

UserID,Fix_Odd_Freq_Aug,Fix_Odd_Stakes_Aug,Fix_Odd_Winnings_Aug,Fix_Odd_Bets_Aug
1324354,16,988.0300,791.6000,25
1324355,8,50.0000,92.5300,15
1324356,12,455.0100,182.7000,42
1324364,4,35.0000,0.0000,3
1324368,7,55.0800,22.5000,8
1324372,10,75.7614,60.2489,28


In [21]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#September
#------------------------------------
#Subset the User Daily Aggregation for Sports book fixed odd product to aggregate by user level
UserDailyAggregation_raw_SBFO<-UserDailyAggregation[ProductID==1,,]
#March
UserDailyAggregation_raw_SBFO_Sep<-UserDailyAggregation_raw_SBFO[Date %between% c("2005-09-01","2005-09-30")]
#Frequency of Play
UserDailyAggregation_SBFO_agg_Sep<-UserDailyAggregation_raw_SBFO_Sep[,Fix_Odd_Freq_Sep:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBFO_agg_Sep<-UserDailyAggregation_raw_SBFO_Sep[,Fix_Odd_Stakes_Sep:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBFO_agg_Sep<-UserDailyAggregation_raw_SBFO_Sep[,Fix_Odd_Winnings_Sep:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBFO_agg_Sep<-UserDailyAggregation_raw_SBFO_Sep[,Fix_Odd_Bets_Sep:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBFO_agg_Sep<-unique(UserDailyAggregation_SBFO_agg_Sep,by="UserID")
UserDailyAggregation_SBFO_agg_Sep<-UserDailyAggregation_SBFO_agg_Sep[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBFO_agg_Feb <- UserDailyAggregation_SBFO_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBFO_agg_Sep)

UserID,Fix_Odd_Freq_Sep,Fix_Odd_Stakes_Sep,Fix_Odd_Winnings_Sep,Fix_Odd_Bets_Sep
1324354,20,1260.9000,1697.7200,60
1324355,15,105.5300,7.3700,42
1324356,6,14.3300,9.0200,5
1324360,10,37.5907,23.1144,17
1324362,4,12.0000,0.0000,5
1324364,2,15.0000,0.0000,2


In [22]:
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_Feb, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_Mar, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_Apr, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_May, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_Jun, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_Jul, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_Aug, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBFO_agg<-merge(x = UserDailyAggregation_SBFO_agg, y = UserDailyAggregation_SBFO_agg_Sep, by = "UserID", all.x = TRUE)
#Droping the unnecessary columns

#cheking the aggregated table result
head(UserDailyAggregation_SBFO_agg)

UserID,Fix_Odd_Dur,Fix_Odd_1Day_Retention,Fix_Odd_Recent,Fix_Odd_Freq_Total,Fix_Odd_Stakes_Total,Fix_Odd_Winnings_Total,Fix_Odd_Bets_Total,Fix_Odd_Freq_FEB,Fix_Odd_Stakes_FEB,...,Fix_Odd_Winnings_Jul,Fix_Odd_Bets_Jul,Fix_Odd_Freq_Aug,Fix_Odd_Stakes_Aug,Fix_Odd_Winnings_Aug,Fix_Odd_Bets_Aug,Fix_Odd_Freq_Sep,Fix_Odd_Stakes_Sep,Fix_Odd_Winnings_Sep,Fix_Odd_Bets_Sep
1324354,218 days,Returned,Recent,117,10137.3300,10224.1200,236,3,40.0000,...,0,0,16,988.03,791.60,25,20,1260.9000,1697.7200,60
1324355,240 days,Returned,Recent,99,400.8600,453.3000,231,21,56.3600,...,NA,NA,8,50.00,92.53,15,15,105.5300,7.3700,42
1324356,222 days,Returned,Not_Recent,51,686.1900,285.5100,98,20,109.9600,...,NA,NA,12,455.01,182.70,42,6,14.3300,9.0200,5
1324358,94 days,Returned,Not_Recent,8,247.6971,153.8756,7,3,120.7323,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324360,235 days,Returned,Recent,29,59.9993,39.9564,40,13,16.6860,...,NA,NA,NA,NA,NA,NA,10,37.5907,23.1144,17
1324362,218 days,Returned,Not_Recent,7,22.0000,0.0000,7,3,10.0000,...,NA,NA,NA,NA,NA,NA,4,12.0000,0.0000,5


In [23]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#First Day of Play
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Dur:=difftime(Live_Act_Last_Date,
                                                            Live_Act_Fisrt_Date,units="days"),by=UserID]
#One Day Retention Calculation
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,
                            Live_Act_1Day_Retention:=.(ifelse(Live_Act_Second_Date==Live_Act_Fisrt_Date+days(1),"Returned","Not_Rerturned")),]
#If customer play the game in last week or not
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,
                            Live_Act_Recent:=.(ifelse(Live_Act_Last_Date>="2005-09-23","Recent","Not_Recent")),]
#Frequency of Play
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Freq_Total:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Stakes_Total:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Winnings_Total:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_raw_SBLA[,Live_Act_Bets_Total:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg<-unique(UserDailyAggregation_SBLA_agg,by="UserID")
#Droping the unnecessary columns
UserDailyAggregation_SBLA_agg<-UserDailyAggregation_SBLA_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,Live_Act_Second_Date=NULL,
                                Live_Act_Fisrt_Date=NULL,Live_Act_Last_Date=NULL),]

head(UserDailyAggregation_SBLA_agg)

UserID,Live_Act_Dur,Live_Act_1Day_Retention,Live_Act_Recent,Live_Act_Freq_Total,Live_Act_Stakes_Total,Live_Act_Winnings_Total,Live_Act_Bets_Total
1324354,145 days,Returned,Recent,19,1839.2800,1512.4900,43
1324355,6 days,Returned,Not_Recent,7,24.7000,11.2000,21
1324356,221 days,Not_Rerturned,Not_Recent,24,679.0700,625.1500,116
1324358,0 days,NA,Not_Recent,1,88.5927,55.9819,4
1324360,221 days,Not_Rerturned,Not_Recent,2,1.7434,1.2009,3
1324364,237 days,Returned,Recent,12,210.0000,40.1500,37


In [24]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#February
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#Calculatinf Frequency and total for February
UserDailyAggregation_raw_SBLA_Feb<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-02-01","2005-02-28")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_Feb<-UserDailyAggregation_raw_SBLA_Feb[,Live_Act_Freq_FEB:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_Feb<-UserDailyAggregation_raw_SBLA_Feb[,Live_Act_Stakes_FEB:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_Feb<-UserDailyAggregation_raw_SBLA_Feb[,Live_Act_Winnings_FEB:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_Feb<-UserDailyAggregation_raw_SBLA_Feb[,Live_Act_Bets_FEB:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_Feb<-unique(UserDailyAggregation_SBLA_agg_Feb,by="UserID")

UserDailyAggregation_SBLA_agg_Feb<-UserDailyAggregation_SBLA_agg_Feb[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_Feb)

UserID,Live_Act_Freq_FEB,Live_Act_Stakes_FEB,Live_Act_Winnings_FEB,Live_Act_Bets_FEB
1324355,7,24.7000,11.2000,21
1324356,7,7.0000,5.8500,13
1324360,1,1.1547,1.2009,2
1324364,5,154.0000,0.0000,23
1324369,21,116.1375,92.1074,85
1324377,1,6.7201,0.0000,4


In [25]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#MARCH
#------------------------------------
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#March
UserDailyAggregation_raw_SBLA_Mar<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-03-01","2005-03-31")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_Mar<-UserDailyAggregation_raw_SBLA_Mar[,Live_Act_Freq_Mar:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_Mar<-UserDailyAggregation_raw_SBLA_Mar[,Live_Act_Stakes_Mar:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_Mar<-UserDailyAggregation_raw_SBLA_Mar[,Live_Act_Winnings_Mar:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_Mar<-UserDailyAggregation_raw_SBLA_Mar[,Live_Act_Bets_Mar:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_Mar<-unique(UserDailyAggregation_SBLA_agg_Mar,by="UserID")
UserDailyAggregation_SBLA_agg_Mar<-UserDailyAggregation_SBLA_agg_Mar[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_Mar)

UserID,Live_Act_Freq_Mar,Live_Act_Stakes_Mar,Live_Act_Winnings_Mar,Live_Act_Bets_Mar
1324356,5,75.5200,60.2300,38
1324364,1,3.0000,0.0000,1
1324368,1,58.8600,49.7600,8
1324369,13,22.2064,22.3947,21
1324372,20,161.1226,148.6056,182
1324404,14,1313.1800,923.3000,26


In [26]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)


#April
#------------------------------------
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#March
UserDailyAggregation_raw_SBLA_Apr<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-04-01","2005-04-30")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_Apr<-UserDailyAggregation_raw_SBLA_Apr[,Live_Act_Freq_Apr:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_Apr<-UserDailyAggregation_raw_SBLA_Apr[,Live_Act_Stakes_Apr:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_Apr<-UserDailyAggregation_raw_SBLA_Apr[,Live_Act_Winnings_Apr:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_Apr<-UserDailyAggregation_raw_SBLA_Apr[,Live_Act_Bets_Apr:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_Apr<-unique(UserDailyAggregation_SBLA_agg_Apr,by="UserID")
UserDailyAggregation_SBLA_agg_Apr<-UserDailyAggregation_SBLA_agg_Apr[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_Apr)

UserID,Live_Act_Freq_Apr,Live_Act_Stakes_Apr,Live_Act_Winnings_Apr,Live_Act_Bets_Apr
1324356,1,0.5000,0.0000,1
1324368,24,2198.1100,2561.6900,146
1324369,1,0.5724,0.8014,1
1324379,2,11.0000,20.0000,2
1324386,2,13.6150,0.0000,3
1324404,15,1453.5000,2012.4300,41


In [27]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#May
#------------------------------------
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#March
UserDailyAggregation_raw_SBLA_May<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-05-01","2005-05-31")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_May<-UserDailyAggregation_raw_SBLA_May[,Live_Act_Freq_May:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_May<-UserDailyAggregation_raw_SBLA_May[,Live_Act_Stakes_May:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_May<-UserDailyAggregation_raw_SBLA_May[,Live_Act_Winnings_May:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_May<-UserDailyAggregation_raw_SBLA_May[,Live_Act_Bets_May:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_May<-unique(UserDailyAggregation_SBLA_agg_May,by="UserID")
UserDailyAggregation_SBLA_agg_May<-UserDailyAggregation_SBLA_agg_May[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_May)

UserID,Live_Act_Freq_May,Live_Act_Stakes_May,Live_Act_Winnings_May,Live_Act_Bets_May
1324354,8,802.9600,616.5600,22
1324358,1,88.5927,55.9819,4
1324368,15,1589.8500,1327.9800,56
1324369,3,8.4482,4.0034,5
1324379,1,20.0000,24.0000,1
1324386,10,137.1642,77.6458,33


In [28]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#June
#------------------------------------
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#March
UserDailyAggregation_raw_SBLA_Jun<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-06-01","2005-06-30")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_Jun<-UserDailyAggregation_raw_SBLA_Jun[,Live_Act_Freq_Jun:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_Jun<-UserDailyAggregation_raw_SBLA_Jun[,Live_Act_Stakes_Jun:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_Jun<-UserDailyAggregation_raw_SBLA_Jun[,Live_Act_Winnings_Jun:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_Jun<-UserDailyAggregation_raw_SBLA_Jun[,Live_Act_Bets_Jun:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_Jun<-unique(UserDailyAggregation_SBLA_agg_Jun,by="UserID")
UserDailyAggregation_SBLA_agg_Jun<-UserDailyAggregation_SBLA_agg_Jun[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_Jun)

UserID,Live_Act_Freq_Jun,Live_Act_Stakes_Jun,Live_Act_Winnings_Jun,Live_Act_Bets_Jun
1324368,7,1100.2900,1182.7300,57
1324379,16,354.2000,421.2900,105
1324386,10,276.8387,246.6623,33
1324404,11,10.7700,10.4400,13
1324416,10,684.7700,516.7700,67
1324449,19,3670.7000,3402.7700,133


In [29]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#July
#------------------------------------
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#March
UserDailyAggregation_raw_SBLA_Jul<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-07-01","2005-07-31")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_Jul<-UserDailyAggregation_raw_SBLA_Jul[,Live_Act_Freq_Jul:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_Jul<-UserDailyAggregation_raw_SBLA_Jul[,Live_Act_Stakes_Jul:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_Jul<-UserDailyAggregation_raw_SBLA_Jul[,Live_Act_Winnings_Jul:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_Jul<-UserDailyAggregation_raw_SBLA_Jul[,Live_Act_Bets_Jul:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_Jul<-unique(UserDailyAggregation_SBLA_agg_Jul,by="UserID")
UserDailyAggregation_SBLA_agg_Jul<-UserDailyAggregation_SBLA_agg_Jul[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_Jul)

UserID,Live_Act_Freq_Jul,Live_Act_Stakes_Jul,Live_Act_Winnings_Jul,Live_Act_Bets_Jul
1324368,9,974.670,818.29,43
1324379,26,1386.673,1290.44,301
1324386,11,2240.517,2077.86,78
1324404,15,542.480,435.32,55
1324416,3,172.500,102.50,16
1324449,20,1646.800,1332.47,100


In [30]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#August
#------------------------------------
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#March
UserDailyAggregation_raw_SBLA_Aug<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-08-01","2005-08-31")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_Aug<-UserDailyAggregation_raw_SBLA_Aug[,Live_Act_Freq_Aug:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_Aug<-UserDailyAggregation_raw_SBLA_Aug[,Live_Act_Stakes_Aug:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_Aug<-UserDailyAggregation_raw_SBLA_Aug[,Live_Act_Winnings_Aug:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_Aug<-UserDailyAggregation_raw_SBLA_Aug[,Live_Act_Bets_Aug:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_Aug<-unique(UserDailyAggregation_SBLA_agg_Aug,by="UserID")
UserDailyAggregation_SBLA_agg_Aug<-UserDailyAggregation_SBLA_agg_Aug[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_Aug)

UserID,Live_Act_Freq_Aug,Live_Act_Stakes_Aug,Live_Act_Winnings_Aug,Live_Act_Bets_Aug
1324354,7,920.2500,866.6800,16
1324356,4,184.3300,167.1400,20
1324364,3,39.0000,40.1500,9
1324368,4,164.5000,134.3900,18
1324372,6,16.2642,12.0626,13
1324379,18,562.5400,600.4600,128


In [31]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#September
#------------------------------------
#Subset the User Daily Aggregation for Live Act to aggregate by user level
UserDailyAggregation_raw_SBLA<-UserDailyAggregation[ProductID==2,,]
#March
UserDailyAggregation_raw_SBLA_Sep<-UserDailyAggregation_raw_SBLA[Date %between% c("2005-09-01","2005-09-30")]
#Frequency of Play
UserDailyAggregation_SBLA_agg_Sep<-UserDailyAggregation_raw_SBLA_Sep[,Live_Act_Freq_Sep:=.N,by=UserID]
#total Stakes
UserDailyAggregation_SBLA_agg_Sep<-UserDailyAggregation_raw_SBLA_Sep[,Live_Act_Stakes_Sep:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_SBLA_agg_Sep<-UserDailyAggregation_raw_SBLA_Sep[,Live_Act_Winnings_Sep:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_SBLA_agg_Sep<-UserDailyAggregation_raw_SBLA_Sep[,Live_Act_Bets_Sep:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_SBLA_agg_Sep<-unique(UserDailyAggregation_SBLA_agg_Sep,by="UserID")
UserDailyAggregation_SBLA_agg_Sep<-UserDailyAggregation_SBLA_agg_Sep[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_SBLA_agg_Feb <- UserDailyAggregation_SBLA_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_SBLA_agg_Sep)

UserID,Live_Act_Freq_Sep,Live_Act_Stakes_Sep,Live_Act_Winnings_Sep,Live_Act_Bets_Sep
1324354,4,116.0700,29.2500,5
1324356,7,411.7200,391.9300,44
1324360,1,0.5887,0.0000,1
1324364,3,14.0000,0.0000,4
1324372,2,25.1124,13.5607,2
1324379,26,1555.3000,1435.9400,331


In [32]:
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_Feb, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_Mar, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_Apr, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_May, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_Jun, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_Jul, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_Aug, by = "UserID", all.x = TRUE)
UserDailyAggregation_SBLA_agg<-merge(x = UserDailyAggregation_SBLA_agg, y = UserDailyAggregation_SBLA_agg_Sep, by = "UserID", all.x = TRUE)
#Droping the unnecessary columns

#cheking the aggregated table result
head(UserDailyAggregation_SBLA_agg)

UserID,Live_Act_Dur,Live_Act_1Day_Retention,Live_Act_Recent,Live_Act_Freq_Total,Live_Act_Stakes_Total,Live_Act_Winnings_Total,Live_Act_Bets_Total,Live_Act_Freq_FEB,Live_Act_Stakes_FEB,...,Live_Act_Winnings_Jul,Live_Act_Bets_Jul,Live_Act_Freq_Aug,Live_Act_Stakes_Aug,Live_Act_Winnings_Aug,Live_Act_Bets_Aug,Live_Act_Freq_Sep,Live_Act_Stakes_Sep,Live_Act_Winnings_Sep,Live_Act_Bets_Sep
1324354,145 days,Returned,Recent,19,1839.2800,1512.4900,43,NA,NA,...,NA,NA,7,920.25,866.68,16,4,116.0700,29.25,5
1324355,6 days,Returned,Not_Recent,7,24.7000,11.2000,21,7,24.7000,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324356,221 days,Not_Rerturned,Not_Recent,24,679.0700,625.1500,116,7,7.0000,...,NA,NA,4,184.33,167.14,20,7,411.7200,391.93,44
1324358,0 days,NA,Not_Recent,1,88.5927,55.9819,4,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324360,221 days,Not_Rerturned,Not_Recent,2,1.7434,1.2009,3,1,1.1547,...,NA,NA,NA,NA,NA,NA,1,0.5887,0.00,1
1324364,237 days,Returned,Recent,12,210.0000,40.1500,37,5,154.0000,...,NA,NA,3,39.00,40.15,9,3,14.0000,0.00,4


In [33]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#First Day of Play
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Dur:=difftime(Casino_Boss_Last_Date,
                                                            Casino_Boss_Fisrt_Date,units="days"),by=UserID]
#One Day Retention Calculation
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,
                            Casino_Boss_1Day_Retention:=.(ifelse(Casino_Boss_Second_Date==Casino_Boss_Fisrt_Date+days(1),"Returned","Not_Rerturned")),]
#If customer play the game in last week or not
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,
                            Casino_Boss_Recent:=.(ifelse(Casino_Boss_Last_Date>="2005-09-23","Recent","Not_Recent")),]
#Frequency of Play
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Freq_Total:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Stakes_Total:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Winnings_Total:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg<-UserDailyAggregation_raw_CBM[,Casino_Boss_Bets_Total:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg<-unique(UserDailyAggregation_CBM_agg,by="UserID")
#Droping the unnecessary columns
UserDailyAggregation_CBM_agg<-UserDailyAggregation_CBM_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,Casino_Boss_Second_Date=NULL,
                                Casino_Boss_Fisrt_Date=NULL,Casino_Boss_Last_Date=NULL),]

head(UserDailyAggregation_CBM_agg)

UserID,Casino_Boss_Dur,Casino_Boss_1Day_Retention,Casino_Boss_Recent,Casino_Boss_Freq_Total,Casino_Boss_Stakes_Total,Casino_Boss_Winnings_Total,Casino_Boss_Bets_Total
1324372,0 days,NA,Not_Recent,1,2.0000,2.0000,12
1324377,0 days,NA,Not_Recent,1,15.0000,4.0000,2
1324379,0 days,NA,Not_Recent,1,0.4499,0.0000,1
1324408,34 days,Not_Rerturned,Not_Recent,2,39.2500,24.5000,3
1324527,0 days,NA,Not_Recent,1,53.7500,53.7759,9
1324675,21 days,Returned,Not_Recent,5,2989.0000,2775.0000,75


In [34]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#February
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#Calculatinf Frequency and total for February
UserDailyAggregation_raw_CBM_Feb<-UserDailyAggregation_raw_CBM[Date %between% c("2005-02-01","2005-02-28")]
#Frequency of Play
UserDailyAggregation_CBM_agg_Feb<-UserDailyAggregation_raw_CBM_Feb[,Casino_Boss_Freq_FEB:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_Feb<-UserDailyAggregation_raw_CBM_Feb[,Casino_Boss_Stakes_FEB:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_Feb<-UserDailyAggregation_raw_CBM_Feb[,Casino_Boss_Winnings_FEB:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_Feb<-UserDailyAggregation_raw_CBM_Feb[,Casino_Boss_Bets_FEB:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_Feb<-unique(UserDailyAggregation_CBM_agg_Feb,by="UserID")

UserDailyAggregation_CBM_agg_Feb<-UserDailyAggregation_CBM_agg_Feb[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_Feb)

UserID,Casino_Boss_Freq_FEB,Casino_Boss_Stakes_FEB,Casino_Boss_Winnings_FEB,Casino_Boss_Bets_FEB
1324408,1,3.00,0.00,1
1324675,5,2989.00,2775.00,75
1324739,1,24.75,15.70,13
1324850,5,944.40,867.55,15
1324858,2,695.00,516.00,2
1325011,1,40.00,34.00,2


In [35]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#MARCH
#------------------------------------
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#March
UserDailyAggregation_raw_CBM_Mar<-UserDailyAggregation_raw_CBM[Date %between% c("2005-03-01","2005-03-31")]
#Frequency of Play
UserDailyAggregation_CBM_agg_Mar<-UserDailyAggregation_raw_CBM_Mar[,Casino_Boss_Freq_Mar:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_Mar<-UserDailyAggregation_raw_CBM_Mar[,Casino_Boss_Stakes_Mar:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_Mar<-UserDailyAggregation_raw_CBM_Mar[,Casino_Boss_Winnings_Mar:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_Mar<-UserDailyAggregation_raw_CBM_Mar[,Casino_Boss_Bets_Mar:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_Mar<-unique(UserDailyAggregation_CBM_agg_Mar,by="UserID")
UserDailyAggregation_CBM_agg_Mar<-UserDailyAggregation_CBM_agg_Mar[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_Mar)

UserID,Casino_Boss_Freq_Mar,Casino_Boss_Stakes_Mar,Casino_Boss_Winnings_Mar,Casino_Boss_Bets_Mar
1324408,1,36.25,24.5,2
1325144,1,14.00,10.0,2
1325161,21,1973.50,1933.0,49
1325189,2,396.00,269.0,5
1325337,4,94.00,91.0,7
1325381,19,11295.19,9785.2,238


In [36]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#April
#------------------------------------
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#March
UserDailyAggregation_raw_CBM_Apr<-UserDailyAggregation_raw_CBM[Date %between% c("2005-04-01","2005-04-30")]
#Frequency of Play
UserDailyAggregation_CBM_agg_Apr<-UserDailyAggregation_raw_CBM_Apr[,Casino_Boss_Freq_Apr:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_Apr<-UserDailyAggregation_raw_CBM_Apr[,Casino_Boss_Stakes_Apr:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_Apr<-UserDailyAggregation_raw_CBM_Apr[,Casino_Boss_Winnings_Apr:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_Apr<-UserDailyAggregation_raw_CBM_Apr[,Casino_Boss_Bets_Apr:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_Apr<-unique(UserDailyAggregation_CBM_agg_Apr,by="UserID")
UserDailyAggregation_CBM_agg_Apr<-UserDailyAggregation_CBM_agg_Apr[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_Apr)

UserID,Casino_Boss_Freq_Apr,Casino_Boss_Stakes_Apr,Casino_Boss_Winnings_Apr,Casino_Boss_Bets_Apr
1324377,1,15.0000,4.000,2
1324379,1,0.4499,0.000,1
1325161,19,2091.0000,2120.500,50
1325189,7,11257.3485,11054.550,30
1325273,2,97.3300,211.838,38
1325337,5,199.0000,184.500,8


In [37]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#May
#------------------------------------
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#March
UserDailyAggregation_raw_CBM_May<-UserDailyAggregation_raw_CBM[Date %between% c("2005-05-01","2005-05-31")]
#Frequency of Play
UserDailyAggregation_CBM_agg_May<-UserDailyAggregation_raw_CBM_May[,Casino_Boss_Freq_May:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_May<-UserDailyAggregation_raw_CBM_May[,Casino_Boss_Stakes_May:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_May<-UserDailyAggregation_raw_CBM_May[,Casino_Boss_Winnings_May:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_May<-UserDailyAggregation_raw_CBM_May[,Casino_Boss_Bets_May:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_May<-unique(UserDailyAggregation_CBM_agg_May,by="UserID")
UserDailyAggregation_CBM_agg_May<-UserDailyAggregation_CBM_agg_May[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_May)

UserID,Casino_Boss_Freq_May,Casino_Boss_Stakes_May,Casino_Boss_Winnings_May,Casino_Boss_Bets_May
1325161,13,1349.000,1364.0,40
1325189,3,290.000,119.0,5
1325337,1,47.000,37.0,1
1325381,12,4835.529,4097.5,38
1325492,2,0.000,0.0,4
1325654,1,3.250,4.0,3


In [38]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#June
#------------------------------------
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#March
UserDailyAggregation_raw_CBM_Jun<-UserDailyAggregation_raw_CBM[Date %between% c("2005-06-01","2005-06-30")]
#Frequency of Play
UserDailyAggregation_CBM_agg_Jun<-UserDailyAggregation_raw_CBM_Jun[,Casino_Boss_Freq_Jun:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_Jun<-UserDailyAggregation_raw_CBM_Jun[,Casino_Boss_Stakes_Jun:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_Jun<-UserDailyAggregation_raw_CBM_Jun[,Casino_Boss_Winnings_Jun:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_Jun<-UserDailyAggregation_raw_CBM_Jun[,Casino_Boss_Bets_Jun:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_Jun<-unique(UserDailyAggregation_CBM_agg_Jun,by="UserID")
UserDailyAggregation_CBM_agg_Jun<-UserDailyAggregation_CBM_agg_Jun[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_Jun)

UserID,Casino_Boss_Freq_Jun,Casino_Boss_Stakes_Jun,Casino_Boss_Winnings_Jun,Casino_Boss_Bets_Jun
1325161,4,211.00,248.00,4
1325189,2,194.00,45.00,2
1325337,1,9.00,0.00,1
1325381,4,3335.70,3044.45,15
1325492,23,6261.75,6008.00,79
1325743,1,20.00,0.00,1


In [39]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#July
#------------------------------------
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#March
UserDailyAggregation_raw_CBM_Jul<-UserDailyAggregation_raw_CBM[Date %between% c("2005-07-01","2005-07-31")]
#Frequency of Play
UserDailyAggregation_CBM_agg_Jul<-UserDailyAggregation_raw_CBM_Jul[,Casino_Boss_Freq_Jul:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_Jul<-UserDailyAggregation_raw_CBM_Jul[,Casino_Boss_Stakes_Jul:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_Jul<-UserDailyAggregation_raw_CBM_Jul[,Casino_Boss_Winnings_Jul:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_Jul<-UserDailyAggregation_raw_CBM_Jul[,Casino_Boss_Bets_Jul:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_Jul<-unique(UserDailyAggregation_CBM_agg_Jul,by="UserID")
UserDailyAggregation_CBM_agg_Jul<-UserDailyAggregation_CBM_agg_Jul[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_Jul)

UserID,Casino_Boss_Freq_Jul,Casino_Boss_Stakes_Jul,Casino_Boss_Winnings_Jul,Casino_Boss_Bets_Jul
1325187,2,60.2500,57.7138,10
1325353,2,262.9600,256.7500,7
1325381,3,310.3993,202.0500,7
1325492,14,2816.0000,2389.2450,29
1326307,5,70345.0000,69525.5000,64
1326311,3,1230.6991,1085.2500,24


In [40]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#August
#------------------------------------
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#March
UserDailyAggregation_raw_CBM_Aug<-UserDailyAggregation_raw_CBM[Date %between% c("2005-08-01","2005-08-31")]
#Frequency of Play
UserDailyAggregation_CBM_agg_Aug<-UserDailyAggregation_raw_CBM_Aug[,Casino_Boss_Freq_Aug:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_Aug<-UserDailyAggregation_raw_CBM_Aug[,Casino_Boss_Stakes_Aug:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_Aug<-UserDailyAggregation_raw_CBM_Aug[,Casino_Boss_Winnings_Aug:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_Aug<-UserDailyAggregation_raw_CBM_Aug[,Casino_Boss_Bets_Aug:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_Aug<-unique(UserDailyAggregation_CBM_agg_Aug,by="UserID")
UserDailyAggregation_CBM_agg_Aug<-UserDailyAggregation_CBM_agg_Aug[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_Aug)

UserID,Casino_Boss_Freq_Aug,Casino_Boss_Stakes_Aug,Casino_Boss_Winnings_Aug,Casino_Boss_Bets_Aug
1324527,1,53.75,53.7759,9
1325187,2,24.00,23.0000,3
1325353,1,167.25,148.2500,4
1325381,1,63.75,17.1000,2
1325453,2,776.95,684.7900,15
1325492,7,0.00,0.0000,11


In [41]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#September
#------------------------------------
#Subset the User Daily Aggregation for Casino Boss to aggregate by user level
UserDailyAggregation_raw_CBM<-UserDailyAggregation[ProductID==4,,]
#March
UserDailyAggregation_raw_CBM_Sep<-UserDailyAggregation_raw_CBM[Date %between% c("2005-09-01","2005-09-30")]
#Frequency of Play
UserDailyAggregation_CBM_agg_Sep<-UserDailyAggregation_raw_CBM_Sep[,Casino_Boss_Freq_Sep:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CBM_agg_Sep<-UserDailyAggregation_raw_CBM_Sep[,Casino_Boss_Stakes_Sep:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CBM_agg_Sep<-UserDailyAggregation_raw_CBM_Sep[,Casino_Boss_Winnings_Sep:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CBM_agg_Sep<-UserDailyAggregation_raw_CBM_Sep[,Casino_Boss_Bets_Sep:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CBM_agg_Sep<-unique(UserDailyAggregation_CBM_agg_Sep,by="UserID")
UserDailyAggregation_CBM_agg_Sep<-UserDailyAggregation_CBM_agg_Sep[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CBM_agg_Feb <- UserDailyAggregation_CBM_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CBM_agg_Sep)

UserID,Casino_Boss_Freq_Sep,Casino_Boss_Stakes_Sep,Casino_Boss_Winnings_Sep,Casino_Boss_Bets_Sep
1324372,1,2.0,2.0,12
1325381,3,565.7,369.4,8
1325492,2,0.0,0.0,2
1325834,5,3970.0,3794.5,26
1326147,1,26.0,34.0,2
1326307,2,2075.0,2070.0,5


In [42]:
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_Feb, by = "UserID", all.x = TRUE)
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_Mar, by = "UserID", all.x = TRUE)
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_Apr, by = "UserID", all.x = TRUE)
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_May, by = "UserID", all.x = TRUE)
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_Jun, by = "UserID", all.x = TRUE)
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_Jul, by = "UserID", all.x = TRUE)
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_Aug, by = "UserID", all.x = TRUE)
UserDailyAggregation_CBM_agg<-merge(x = UserDailyAggregation_CBM_agg, y = UserDailyAggregation_CBM_agg_Sep, by = "UserID", all.x = TRUE)
#Droping the unnecessary columns

#cheking the aggregated table result
head(UserDailyAggregation_CBM_agg)

UserID,Casino_Boss_Dur,Casino_Boss_1Day_Retention,Casino_Boss_Recent,Casino_Boss_Freq_Total,Casino_Boss_Stakes_Total,Casino_Boss_Winnings_Total,Casino_Boss_Bets_Total,Casino_Boss_Freq_FEB,Casino_Boss_Stakes_FEB,...,Casino_Boss_Winnings_Jul,Casino_Boss_Bets_Jul,Casino_Boss_Freq_Aug,Casino_Boss_Stakes_Aug,Casino_Boss_Winnings_Aug,Casino_Boss_Bets_Aug,Casino_Boss_Freq_Sep,Casino_Boss_Stakes_Sep,Casino_Boss_Winnings_Sep,Casino_Boss_Bets_Sep
1324372,0 days,NA,Not_Recent,1,2.0000,2.0000,12,NA,NA,...,NA,NA,NA,NA,NA,NA,1,2,2,12
1324377,0 days,NA,Not_Recent,1,15.0000,4.0000,2,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324379,0 days,NA,Not_Recent,1,0.4499,0.0000,1,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324408,34 days,Not_Rerturned,Not_Recent,2,39.2500,24.5000,3,1,3,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324527,0 days,NA,Not_Recent,1,53.7500,53.7759,9,NA,NA,...,NA,NA,1,53.75,53.7759,9,NA,NA,NA,NA
1324675,21 days,Returned,Not_Recent,5,2989.0000,2775.0000,75,5,2989,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [43]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#First Day of Play
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Dur:=difftime(Supertoto_Last_Date,
                                                            Supertoto_Fisrt_Date,units="days"),by=UserID]
#One Day Retention Calculation
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,
                            Supertoto_1Day_Retention:=.(ifelse(Supertoto_Second_Date==Supertoto_Fisrt_Date+days(1),"Returned","Not_Rerturned")),]
#If customer play the game in last week or not
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,
                            Supertoto_Recent:=.(ifelse(Supertoto_Last_Date>="2005-09-23","Recent","Not_Recent")),]
#Frequency of Play
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Freq_Total:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Stakes_Total:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Winnings_Total:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_raw_Supertoto[,Supertoto_Bets_Total:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg<-unique(UserDailyAggregation_Supertoto_agg,by="UserID")
#Droping the unnecessary columns
UserDailyAggregation_Supertoto_agg<-UserDailyAggregation_Supertoto_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,Supertoto_Second_Date=NULL,
                                Supertoto_Fisrt_Date=NULL,Supertoto_Last_Date=NULL),]

head(UserDailyAggregation_Supertoto_agg)

UserID,Supertoto_Dur,Supertoto_1Day_Retention,Supertoto_Recent,Supertoto_Freq_Total,Supertoto_Stakes_Total,Supertoto_Winnings_Total,Supertoto_Bets_Total
1324386,107 days,Not_Rerturned,Not_Recent,5,4.6,3.8150,7
1324454,20 days,Returned,Not_Recent,10,32.1,26.4825,16
1324527,0 days,NA,Not_Recent,1,5.5,4.5375,2
1324708,121 days,Not_Rerturned,Not_Recent,32,31.5,26.0275,67
1324866,28 days,Returned,Not_Recent,10,8.1,6.7625,35
1324868,0 days,NA,Not_Recent,1,0.1,0.0825,1


In [44]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#February
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#Calculatinf Frequency and total for February
UserDailyAggregation_raw_Supertoto_Feb<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-02-01","2005-02-28")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_Feb<-UserDailyAggregation_raw_Supertoto_Feb[,Supertoto_Freq_FEB:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_Feb<-UserDailyAggregation_raw_Supertoto_Feb[,Supertoto_Stakes_FEB:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_Feb<-UserDailyAggregation_raw_Supertoto_Feb[,Supertoto_Winnings_FEB:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_Feb<-UserDailyAggregation_raw_Supertoto_Feb[,Supertoto_Bets_FEB:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_Feb<-unique(UserDailyAggregation_Supertoto_agg_Feb,by="UserID")

UserDailyAggregation_Supertoto_agg_Feb<-UserDailyAggregation_Supertoto_agg_Feb[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_Feb)

UserID,Supertoto_Freq_FEB,Supertoto_Stakes_FEB,Supertoto_Winnings_FEB,Supertoto_Bets_FEB
1324454,7,27.8,22.9350,12
1324708,11,12.7,10.4775,18
1325077,2,3.7,3.3725,2
1325080,2,33.6,27.7200,2
1325337,1,0.5,0.4125,1
1325491,2,10.1,8.3325,101


In [45]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#MARCH
#------------------------------------
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#March
UserDailyAggregation_raw_Supertoto_Mar<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-03-01","2005-03-31")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_Mar<-UserDailyAggregation_raw_Supertoto_Mar[,Supertoto_Freq_Mar:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_Mar<-UserDailyAggregation_raw_Supertoto_Mar[,Supertoto_Stakes_Mar:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_Mar<-UserDailyAggregation_raw_Supertoto_Mar[,Supertoto_Winnings_Mar:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_Mar<-UserDailyAggregation_raw_Supertoto_Mar[,Supertoto_Bets_Mar:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_Mar<-unique(UserDailyAggregation_Supertoto_agg_Mar,by="UserID")
UserDailyAggregation_Supertoto_agg_Mar<-UserDailyAggregation_Supertoto_agg_Mar[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_Mar)

UserID,Supertoto_Freq_Mar,Supertoto_Stakes_Mar,Supertoto_Winnings_Mar,Supertoto_Bets_Mar
1324454,3,4.3,3.5475,4
1324708,12,10.9,9.0025,35
1325491,2,5.4,4.4550,54
1325619,8,32.5,27.1725,6
1325703,4,9.7,8.0025,6
1325824,1,0.1,0.0825,1


In [46]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#April
#------------------------------------
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#March
UserDailyAggregation_raw_Supertoto_Apr<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-04-01","2005-04-30")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_Apr<-UserDailyAggregation_raw_Supertoto_Apr[,Supertoto_Freq_Apr:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_Apr<-UserDailyAggregation_raw_Supertoto_Apr[,Supertoto_Stakes_Apr:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_Apr<-UserDailyAggregation_raw_Supertoto_Apr[,Supertoto_Winnings_Apr:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_Apr<-UserDailyAggregation_raw_Supertoto_Apr[,Supertoto_Bets_Apr:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_Apr<-unique(UserDailyAggregation_Supertoto_agg_Apr,by="UserID")
UserDailyAggregation_Supertoto_agg_Apr<-UserDailyAggregation_Supertoto_agg_Apr[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_Apr)

UserID,Supertoto_Freq_Apr,Supertoto_Stakes_Apr,Supertoto_Winnings_Apr,Supertoto_Bets_Apr
1324386,1,0.2,0.1850,2
1324708,4,5.0,4.1250,9
1325024,1,3.2,2.6400,1
1325080,5,36.6,30.1950,4
1325619,1,0.1,0.0825,1
1325703,1,0.8,0.6600,1


In [47]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#May
#------------------------------------
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#March
UserDailyAggregation_raw_Supertoto_May<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-05-01","2005-05-31")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_May<-UserDailyAggregation_raw_Supertoto_May[,Supertoto_Freq_May:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_May<-UserDailyAggregation_raw_Supertoto_May[,Supertoto_Stakes_May:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_May<-UserDailyAggregation_raw_Supertoto_May[,Supertoto_Winnings_May:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_May<-UserDailyAggregation_raw_Supertoto_May[,Supertoto_Bets_May:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_May<-unique(UserDailyAggregation_Supertoto_agg_May,by="UserID")
UserDailyAggregation_Supertoto_agg_May<-UserDailyAggregation_Supertoto_agg_May[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_May)

UserID,Supertoto_Freq_May,Supertoto_Stakes_May,Supertoto_Winnings_May,Supertoto_Bets_May
1324708,3,2.1,1.7325,3
1324868,1,0.1,0.0825,1
1325024,2,3.2,2.6400,2
1325080,3,30.4,25.0800,2
1325337,1,4.0,3.3000,1
1325619,1,0.2,0.1650,2


In [48]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#June
#------------------------------------
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#March
UserDailyAggregation_raw_Supertoto_Jun<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-06-01","2005-06-30")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_Jun<-UserDailyAggregation_raw_Supertoto_Jun[,Supertoto_Freq_Jun:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_Jun<-UserDailyAggregation_raw_Supertoto_Jun[,Supertoto_Stakes_Jun:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_Jun<-UserDailyAggregation_raw_Supertoto_Jun[,Supertoto_Winnings_Jun:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_Jun<-UserDailyAggregation_raw_Supertoto_Jun[,Supertoto_Bets_Jun:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_Jun<-unique(UserDailyAggregation_Supertoto_agg_Jun,by="UserID")
UserDailyAggregation_Supertoto_agg_Jun<-UserDailyAggregation_Supertoto_agg_Jun[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_Jun)

UserID,Supertoto_Freq_Jun,Supertoto_Stakes_Jun,Supertoto_Winnings_Jun,Supertoto_Bets_Jun
1324386,1,3.2,2.6400,1
1324708,2,0.8,0.6900,2
1324866,5,4.4,3.6300,14
1325706,1,0.8,0.6600,1
1326427,1,0.1,0.0825,1
1326592,1,2.0,1.6500,1


In [49]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#July
#------------------------------------
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#March
UserDailyAggregation_raw_Supertoto_Jul<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-07-01","2005-07-31")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_Jul<-UserDailyAggregation_raw_Supertoto_Jul[,Supertoto_Freq_Jul:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_Jul<-UserDailyAggregation_raw_Supertoto_Jul[,Supertoto_Stakes_Jul:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_Jul<-UserDailyAggregation_raw_Supertoto_Jul[,Supertoto_Winnings_Jul:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_Jul<-UserDailyAggregation_raw_Supertoto_Jul[,Supertoto_Bets_Jul:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_Jul<-unique(UserDailyAggregation_Supertoto_agg_Jul,by="UserID")
UserDailyAggregation_Supertoto_agg_Jul<-UserDailyAggregation_Supertoto_agg_Jul[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_Jul)

UserID,Supertoto_Freq_Jul,Supertoto_Stakes_Jul,Supertoto_Winnings_Jul,Supertoto_Bets_Jul
1324866,5,3.7,3.1325,21
1325027,7,16.5,14.2525,6
1325337,1,7.2,6.6600,1
1326452,1,6.4,5.2800,1
1327179,1,0.5,0.4125,1
1327419,2,4.0,3.3000,1


In [50]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#August
#------------------------------------
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#March
UserDailyAggregation_raw_Supertoto_Aug<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-08-01","2005-08-31")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_Aug<-UserDailyAggregation_raw_Supertoto_Aug[,Supertoto_Freq_Aug:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_Aug<-UserDailyAggregation_raw_Supertoto_Aug[,Supertoto_Stakes_Aug:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_Aug<-UserDailyAggregation_raw_Supertoto_Aug[,Supertoto_Winnings_Aug:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_Aug<-UserDailyAggregation_raw_Supertoto_Aug[,Supertoto_Bets_Aug:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_Aug<-unique(UserDailyAggregation_Supertoto_agg_Aug,by="UserID")
UserDailyAggregation_Supertoto_agg_Aug<-UserDailyAggregation_Supertoto_agg_Aug[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_Aug)

UserID,Supertoto_Freq_Aug,Supertoto_Stakes_Aug,Supertoto_Winnings_Aug,Supertoto_Bets_Aug
1324386,3,1.2,0.9900,4
1324527,1,5.5,4.5375,2
1325027,4,7.9,6.5175,4
1325619,2,1.0,0.8250,2
1326085,1,0.1,0.0925,1
1326452,2,3.8,3.1350,4


In [51]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#September
#------------------------------------
#Subset the User Daily Aggregation for SuperToTo to aggregate by user level
UserDailyAggregation_raw_Supertoto<-UserDailyAggregation[ProductID==5,,]
#March
UserDailyAggregation_raw_Supertoto_Sep<-UserDailyAggregation_raw_Supertoto[Date %between% c("2005-09-01","2005-09-30")]
#Frequency of Play
UserDailyAggregation_Supertoto_agg_Sep<-UserDailyAggregation_raw_Supertoto_Sep[,Supertoto_Freq_Sep:=.N,by=UserID]
#total Stakes
UserDailyAggregation_Supertoto_agg_Sep<-UserDailyAggregation_raw_Supertoto_Sep[,Supertoto_Stakes_Sep:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_Supertoto_agg_Sep<-UserDailyAggregation_raw_Supertoto_Sep[,Supertoto_Winnings_Sep:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_Supertoto_agg_Sep<-UserDailyAggregation_raw_Supertoto_Sep[,Supertoto_Bets_Sep:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_Supertoto_agg_Sep<-unique(UserDailyAggregation_Supertoto_agg_Sep,by="UserID")
UserDailyAggregation_Supertoto_agg_Sep<-UserDailyAggregation_Supertoto_agg_Sep[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_Supertoto_agg_Feb <- UserDailyAggregation_Supertoto_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_Supertoto_agg_Sep)

UserID,Supertoto_Freq_Sep,Supertoto_Stakes_Sep,Supertoto_Winnings_Sep,Supertoto_Bets_Sep
1325337,4,13.2,10.8900,4
1325619,3,2.5,2.0625,4
1326070,1,0.5,0.4125,1
1326592,2,16.8,13.8600,2
1326788,1,1.0,0.8250,1
1326907,5,1.9,1.5675,5


In [52]:
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_Feb, by = "UserID", all.x = TRUE)
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_Mar, by = "UserID", all.x = TRUE)
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_Apr, by = "UserID", all.x = TRUE)
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_May, by = "UserID", all.x = TRUE)
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_Jun, by = "UserID", all.x = TRUE)
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_Jul, by = "UserID", all.x = TRUE)
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_Aug, by = "UserID", all.x = TRUE)
UserDailyAggregation_Supertoto_agg<-merge(x = UserDailyAggregation_Supertoto_agg, y = UserDailyAggregation_Supertoto_agg_Sep, by = "UserID", all.x = TRUE)
#Droping the unnecessary columns

#cheking the aggregated table result
head(UserDailyAggregation_Supertoto_agg)

UserID,Supertoto_Dur,Supertoto_1Day_Retention,Supertoto_Recent,Supertoto_Freq_Total,Supertoto_Stakes_Total,Supertoto_Winnings_Total,Supertoto_Bets_Total,Supertoto_Freq_FEB,Supertoto_Stakes_FEB,...,Supertoto_Winnings_Jul,Supertoto_Bets_Jul,Supertoto_Freq_Aug,Supertoto_Stakes_Aug,Supertoto_Winnings_Aug,Supertoto_Bets_Aug,Supertoto_Freq_Sep,Supertoto_Stakes_Sep,Supertoto_Winnings_Sep,Supertoto_Bets_Sep
1324386,107 days,Not_Rerturned,Not_Recent,5,4.6,3.8150,7,NA,NA,...,NA,NA,3,1.2,0.9900,4,NA,NA,NA,NA
1324454,20 days,Returned,Not_Recent,10,32.1,26.4825,16,7,27.8,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324527,0 days,NA,Not_Recent,1,5.5,4.5375,2,NA,NA,...,NA,NA,1,5.5,4.5375,2,NA,NA,NA,NA
1324708,121 days,Not_Rerturned,Not_Recent,32,31.5,26.0275,67,11,12.7,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324866,28 days,Returned,Not_Recent,10,8.1,6.7625,35,NA,NA,...,3.1325,21,NA,NA,NA,NA,NA,NA,NA,NA
1324868,0 days,NA,Not_Recent,1,0.1,0.0825,1,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [53]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#First Day of Play
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Dur:=difftime(Games_VS_Last_Date,
                                                            Games_VS_Fisrt_Date,units="days"),by=UserID]
#One Day Retention Calculation
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,
                            Games_VS_1Day_Retention:=.(ifelse(Games_VS_Second_Date==Games_VS_Fisrt_Date+days(1),"Returned","Not_Rerturned")),]
#If customer play the game in last week or not
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,
                            Games_VS_Recent:=.(ifelse(Games_VS_Last_Date>="2005-09-23","Recent","Not_Recent")),]
#Frequency of Play
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Freq_Total:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Stakes_Total:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Winnings_Total:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg<-UserDailyAggregation_raw_GVS[,Games_VS_Bets_Total:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg<-unique(UserDailyAggregation_GVS_agg,by="UserID")
#Droping the unnecessary columns
UserDailyAggregation_GVS_agg<-UserDailyAggregation_GVS_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,Games_VS_Second_Date=NULL,
                                Games_VS_Fisrt_Date=NULL,Games_VS_Last_Date=NULL),]

head(UserDailyAggregation_GVS_agg)

UserID,Games_VS_Dur,Games_VS_1Day_Retention,Games_VS_Recent,Games_VS_Freq_Total,Games_VS_Stakes_Total,Games_VS_Winnings_Total,Games_VS_Bets_Total
1324368,0 days,NA,Not_Recent,1,1.0000,0.0000,1
1324369,0 days,NA,Not_Recent,1,8.8496,4.8968,7
1324372,1 days,Returned,Not_Recent,2,18.5832,18.0809,19
1324383,0 days,NA,Not_Recent,1,22.1043,2.0374,6
1324386,7 days,Returned,Recent,3,268.8806,224.5812,136
1324405,160 days,Returned,Not_Recent,7,112.2900,111.1500,106


In [54]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#February
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#Calculatinf Frequency and total for February
UserDailyAggregation_raw_GVS_Feb<-UserDailyAggregation_raw_GVS[Date %between% c("2005-02-01","2005-02-28")]
#Frequency of Play
UserDailyAggregation_GVS_agg_Feb<-UserDailyAggregation_raw_GVS_Feb[,Games_VS_Freq_FEB:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_Feb<-UserDailyAggregation_raw_GVS_Feb[,Games_VS_Stakes_FEB:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_Feb<-UserDailyAggregation_raw_GVS_Feb[,Games_VS_Winnings_FEB:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_Feb<-UserDailyAggregation_raw_GVS_Feb[,Games_VS_Bets_FEB:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_Feb<-unique(UserDailyAggregation_GVS_agg_Feb,by="UserID")

UserDailyAggregation_GVS_agg_Feb<-UserDailyAggregation_GVS_agg_Feb[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_Feb)

UserID,Games_VS_Freq_FEB,Games_VS_Stakes_FEB,Games_VS_Winnings_FEB,Games_VS_Bets_FEB
1324405,4,109.2900,107.6900,100
1324408,1,2.0000,0.0000,1
1324418,1,1.1547,1.1547,1
1324432,4,31.9342,19.9423,23
1324437,2,19.6305,21.0970,17
1324454,9,401.1700,324.2900,273


In [55]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#MARCH
#------------------------------------
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#March
UserDailyAggregation_raw_GVS_Mar<-UserDailyAggregation_raw_GVS[Date %between% c("2005-03-01","2005-03-31")]
#Frequency of Play
UserDailyAggregation_GVS_agg_Mar<-UserDailyAggregation_raw_GVS_Mar[,Games_VS_Freq_Mar:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_Mar<-UserDailyAggregation_raw_GVS_Mar[,Games_VS_Stakes_Mar:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_Mar<-UserDailyAggregation_raw_GVS_Mar[,Games_VS_Winnings_Mar:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_Mar<-UserDailyAggregation_raw_GVS_Mar[,Games_VS_Bets_Mar:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_Mar<-unique(UserDailyAggregation_GVS_agg_Mar,by="UserID")
UserDailyAggregation_GVS_agg_Mar<-UserDailyAggregation_GVS_agg_Mar[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_Mar)

UserID,Games_VS_Freq_Mar,Games_VS_Stakes_Mar,Games_VS_Winnings_Mar,Games_VS_Bets_Mar
1324369,1,8.8496,4.8968,7
1324405,1,0.5000,0.0000,1
1324454,8,671.3600,537.5700,290
1324495,1,3.0000,0.0000,1
1324615,3,750.5000,621.1900,290
1325165,8,2185.8300,1910.4900,598


In [56]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#April
#------------------------------------
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#March
UserDailyAggregation_raw_GVS_Apr<-UserDailyAggregation_raw_GVS[Date %between% c("2005-04-01","2005-04-30")]
#Frequency of Play
UserDailyAggregation_GVS_agg_Apr<-UserDailyAggregation_raw_GVS_Apr[,Games_VS_Freq_Apr:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_Apr<-UserDailyAggregation_raw_GVS_Apr[,Games_VS_Stakes_Apr:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_Apr<-UserDailyAggregation_raw_GVS_Apr[,Games_VS_Winnings_Apr:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_Apr<-UserDailyAggregation_raw_GVS_Apr[,Games_VS_Bets_Apr:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_Apr<-unique(UserDailyAggregation_GVS_agg_Apr,by="UserID")
UserDailyAggregation_GVS_agg_Apr<-UserDailyAggregation_GVS_agg_Apr[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_Apr)

UserID,Games_VS_Freq_Apr,Games_VS_Stakes_Apr,Games_VS_Winnings_Apr,Games_VS_Bets_Apr
1324749,1,9.0000,4.0000,9
1325080,1,36.0000,30.2700,4
1325106,3,38.9252,39.0298,26
1325427,1,4.0000,0.0000,1
1325431,8,284.7454,284.0972,179
1325664,1,18.7500,12.5000,16


In [57]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#May
#------------------------------------
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#March
UserDailyAggregation_raw_GVS_May<-UserDailyAggregation_raw_GVS[Date %between% c("2005-05-01","2005-05-31")]
#Frequency of Play
UserDailyAggregation_GVS_agg_May<-UserDailyAggregation_raw_GVS_May[,Games_VS_Freq_May:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_May<-UserDailyAggregation_raw_GVS_May[,Games_VS_Stakes_May:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_May<-UserDailyAggregation_raw_GVS_May[,Games_VS_Winnings_May:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_May<-UserDailyAggregation_raw_GVS_May[,Games_VS_Bets_May:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_May<-unique(UserDailyAggregation_GVS_agg_May,by="UserID")
UserDailyAggregation_GVS_agg_May<-UserDailyAggregation_GVS_agg_May[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_May)

UserID,Games_VS_Freq_May,Games_VS_Stakes_May,Games_VS_Winnings_May,Games_VS_Bets_May
1324644,1,42.2900,13.3900,9
1324868,2,913.6808,739.4736,136
1325249,3,637.2045,587.1149,142
1325431,4,7.3872,4.8971,10
1325654,2,443.5201,364.9382,640
1325694,1,3.3613,0.0000,3


In [58]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#June
#------------------------------------
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#March
UserDailyAggregation_raw_GVS_Jun<-UserDailyAggregation_raw_GVS[Date %between% c("2005-06-01","2005-06-30")]
#Frequency of Play
UserDailyAggregation_GVS_agg_Jun<-UserDailyAggregation_raw_GVS_Jun[,Games_VS_Freq_Jun:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_Jun<-UserDailyAggregation_raw_GVS_Jun[,Games_VS_Stakes_Jun:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_Jun<-UserDailyAggregation_raw_GVS_Jun[,Games_VS_Winnings_Jun:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_Jun<-UserDailyAggregation_raw_GVS_Jun[,Games_VS_Bets_Jun:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_Jun<-unique(UserDailyAggregation_GVS_agg_Jun,by="UserID")
UserDailyAggregation_GVS_agg_Jun<-UserDailyAggregation_GVS_agg_Jun[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_Jun)

UserID,Games_VS_Freq_Jun,Games_VS_Stakes_Jun,Games_VS_Winnings_Jun,Games_VS_Bets_Jun
1324383,1,22.1043,2.0374,6
1324803,6,449.8207,432.2892,197
1325027,1,28.9500,16.0000,16
1325148,1,31.3000,16.6400,31
1325187,1,3.3636,3.3636,7
1325636,1,0.9814,0.0000,1


In [59]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#July
#------------------------------------
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#March
UserDailyAggregation_raw_GVS_Jul<-UserDailyAggregation_raw_GVS[Date %between% c("2005-07-01","2005-07-31")]
#Frequency of Play
UserDailyAggregation_GVS_agg_Jul<-UserDailyAggregation_raw_GVS_Jul[,Games_VS_Freq_Jul:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_Jul<-UserDailyAggregation_raw_GVS_Jul[,Games_VS_Stakes_Jul:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_Jul<-UserDailyAggregation_raw_GVS_Jul[,Games_VS_Winnings_Jul:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_Jul<-UserDailyAggregation_raw_GVS_Jul[,Games_VS_Bets_Jul:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_Jul<-unique(UserDailyAggregation_GVS_agg_Jul,by="UserID")
UserDailyAggregation_GVS_agg_Jul<-UserDailyAggregation_GVS_agg_Jul[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_Jul)

UserID,Games_VS_Freq_Jul,Games_VS_Stakes_Jul,Games_VS_Winnings_Jul,Games_VS_Bets_Jul
1324405,1,1.5000,2.8700,3
1324803,4,137.2366,122.8005,30
1324808,1,517.4856,482.9940,522
1325126,2,118.4600,90.2800,107
1325148,3,91.5100,69.7300,67
1325165,1,1432.0700,1242.7000,64


In [60]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#August
#------------------------------------
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#March
UserDailyAggregation_raw_GVS_Aug<-UserDailyAggregation_raw_GVS[Date %between% c("2005-08-01","2005-08-31")]
#Frequency of Play
UserDailyAggregation_GVS_agg_Aug<-UserDailyAggregation_raw_GVS_Aug[,Games_VS_Freq_Aug:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_Aug<-UserDailyAggregation_raw_GVS_Aug[,Games_VS_Stakes_Aug:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_Aug<-UserDailyAggregation_raw_GVS_Aug[,Games_VS_Winnings_Aug:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_Aug<-UserDailyAggregation_raw_GVS_Aug[,Games_VS_Bets_Aug:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_Aug<-unique(UserDailyAggregation_GVS_agg_Aug,by="UserID")
UserDailyAggregation_GVS_agg_Aug<-UserDailyAggregation_GVS_agg_Aug[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_Aug)

UserID,Games_VS_Freq_Aug,Games_VS_Stakes_Aug,Games_VS_Winnings_Aug,Games_VS_Bets_Aug
1324368,1,1.0000,0.0000,1
1324405,1,1.0000,0.5900,2
1324803,2,43.3759,27.3454,11
1325148,2,15.6300,10.3300,10
1325200,1,79.9356,35.3204,15
1325243,1,0.5000,0.0000,1


In [61]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#September
#------------------------------------
#Subset the User Daily Aggregation for G V S to aggregate by user level
UserDailyAggregation_raw_GVS<-UserDailyAggregation[ProductID==6,,]
#March
UserDailyAggregation_raw_GVS_Sep<-UserDailyAggregation_raw_GVS[Date %between% c("2005-09-01","2005-09-30")]
#Frequency of Play
UserDailyAggregation_GVS_agg_Sep<-UserDailyAggregation_raw_GVS_Sep[,Games_VS_Freq_Sep:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GVS_agg_Sep<-UserDailyAggregation_raw_GVS_Sep[,Games_VS_Stakes_Sep:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GVS_agg_Sep<-UserDailyAggregation_raw_GVS_Sep[,Games_VS_Winnings_Sep:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GVS_agg_Sep<-UserDailyAggregation_raw_GVS_Sep[,Games_VS_Bets_Sep:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GVS_agg_Sep<-unique(UserDailyAggregation_GVS_agg_Sep,by="UserID")
UserDailyAggregation_GVS_agg_Sep<-UserDailyAggregation_GVS_agg_Sep[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GVS_agg_Feb <- UserDailyAggregation_GVS_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GVS_agg_Sep)

UserID,Games_VS_Freq_Sep,Games_VS_Stakes_Sep,Games_VS_Winnings_Sep,Games_VS_Bets_Sep
1324372,2,18.5832,18.0809,19
1324386,3,268.8806,224.5812,136
1325165,1,0.5000,0.0000,1
1325215,3,16.7597,20.4666,11
1325870,2,3.5158,2.7323,9
1326028,1,51.0000,17.1600,8


In [62]:
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_Feb, by = "UserID", all.x = TRUE)
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_Mar, by = "UserID", all.x = TRUE)
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_Apr, by = "UserID", all.x = TRUE)
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_May, by = "UserID", all.x = TRUE)
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_Jun, by = "UserID", all.x = TRUE)
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_Jul, by = "UserID", all.x = TRUE)
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_Aug, by = "UserID", all.x = TRUE)
UserDailyAggregation_GVS_agg<-merge(x = UserDailyAggregation_GVS_agg, y = UserDailyAggregation_GVS_agg_Sep, by = "UserID", all.x = TRUE)
#Droping the unnecessary columns

#cheking the aggregated table result
head(UserDailyAggregation_GVS_agg)

UserID,Games_VS_Dur,Games_VS_1Day_Retention,Games_VS_Recent,Games_VS_Freq_Total,Games_VS_Stakes_Total,Games_VS_Winnings_Total,Games_VS_Bets_Total,Games_VS_Freq_FEB,Games_VS_Stakes_FEB,...,Games_VS_Winnings_Jul,Games_VS_Bets_Jul,Games_VS_Freq_Aug,Games_VS_Stakes_Aug,Games_VS_Winnings_Aug,Games_VS_Bets_Aug,Games_VS_Freq_Sep,Games_VS_Stakes_Sep,Games_VS_Winnings_Sep,Games_VS_Bets_Sep
1324368,0 days,NA,Not_Recent,1,1.0000,0.0000,1,NA,NA,...,NA,NA,1,1,0.00,1,NA,NA,NA,NA
1324369,0 days,NA,Not_Recent,1,8.8496,4.8968,7,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324372,1 days,Returned,Not_Recent,2,18.5832,18.0809,19,NA,NA,...,NA,NA,NA,NA,NA,NA,2,18.5832,18.0809,19
1324383,0 days,NA,Not_Recent,1,22.1043,2.0374,6,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324386,7 days,Returned,Recent,3,268.8806,224.5812,136,NA,NA,...,NA,NA,NA,NA,NA,NA,3,268.8806,224.5812,136
1324405,160 days,Returned,Not_Recent,7,112.2900,111.1500,106,4,109.29,...,2.87,3,1,1,0.59,2,NA,NA,NA,NA


In [63]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#First Day of Play
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Dur:=difftime(Games_Bwin_Last_Date,
                                                            Games_Bwin_Fisrt_Date,units="days"),by=UserID]
#One Day Retention Calculation
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,
                            Games_Bwin_1Day_Retention:=.(ifelse(Games_Bwin_Second_Date==Games_Bwin_Fisrt_Date+days(1),"Returned","Not_Rerturned")),]
#If customer play the game in last week or not
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,
                            Games_Bwin_Recent:=.(ifelse(Games_Bwin_Last_Date>="2005-09-23","Recent","Not_Recent")),]
#Frequency of Play
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Freq_Total:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Stakes_Total:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Winnings_Total:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg<-UserDailyAggregation_raw_GB[,Games_Bwin_Bets_Total:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg<-unique(UserDailyAggregation_GB_agg,by="UserID")
#Droping the unnecessary columns
UserDailyAggregation_GB_agg<-UserDailyAggregation_GB_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,Games_Bwin_Second_Date=NULL,
                                Games_Bwin_Fisrt_Date=NULL,Games_Bwin_Last_Date=NULL),]

head(UserDailyAggregation_GB_agg)

UserID,Games_Bwin_Dur,Games_Bwin_1Day_Retention,Games_Bwin_Recent,Games_Bwin_Freq_Total,Games_Bwin_Stakes_Total,Games_Bwin_Winnings_Total,Games_Bwin_Bets_Total
1324369,11 days,Not_Rerturned,Not_Recent,2,2.00,0.000,2
1324379,133 days,Not_Rerturned,Not_Recent,12,1009.63,938.349,425
1324408,0 days,NA,Not_Recent,1,1.00,0.500,2
1324437,6 days,Returned,Not_Recent,6,866.00,716.000,215
1324454,28 days,Not_Rerturned,Not_Recent,4,28.10,8.200,32
1324615,21 days,Not_Rerturned,Not_Recent,3,44.50,31.500,72


In [64]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#February
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#Calculatinf Frequency and total for February
UserDailyAggregation_raw_GB_Feb<-UserDailyAggregation_raw_GB[Date %between% c("2005-02-01","2005-02-28")]
#Frequency of Play
UserDailyAggregation_GB_agg_Feb<-UserDailyAggregation_raw_GB_Feb[,Games_Bwin_Freq_FEB:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_Feb<-UserDailyAggregation_raw_GB_Feb[,Games_Bwin_Stakes_FEB:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_Feb<-UserDailyAggregation_raw_GB_Feb[,Games_Bwin_Winnings_FEB:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_Feb<-UserDailyAggregation_raw_GB_Feb[,Games_Bwin_Bets_FEB:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_Feb<-unique(UserDailyAggregation_GB_agg_Feb,by="UserID")

UserDailyAggregation_GB_agg_Feb<-UserDailyAggregation_GB_agg_Feb[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_Feb)

UserID,Games_Bwin_Freq_FEB,Games_Bwin_Stakes_FEB,Games_Bwin_Winnings_FEB,Games_Bwin_Bets_FEB
1324369,2,2.00,0.00,2
1324408,1,1.00,0.50,2
1324437,6,866.00,716.00,215
1324454,3,20.25,5.25,24
1324624,1,3.50,1.50,6
1324732,2,263.00,243.00,519


In [65]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#MARCH
#------------------------------------
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#March
UserDailyAggregation_raw_GB_Mar<-UserDailyAggregation_raw_GB[Date %between% c("2005-03-01","2005-03-31")]
#Frequency of Play
UserDailyAggregation_GB_agg_Mar<-UserDailyAggregation_raw_GB_Mar[,Games_Bwin_Freq_Mar:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_Mar<-UserDailyAggregation_raw_GB_Mar[,Games_Bwin_Stakes_Mar:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_Mar<-UserDailyAggregation_raw_GB_Mar[,Games_Bwin_Winnings_Mar:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_Mar<-UserDailyAggregation_raw_GB_Mar[,Games_Bwin_Bets_Mar:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_Mar<-unique(UserDailyAggregation_GB_agg_Mar,by="UserID")
UserDailyAggregation_GB_agg_Mar<-UserDailyAggregation_GB_agg_Mar[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_Mar)

UserID,Games_Bwin_Freq_Mar,Games_Bwin_Stakes_Mar,Games_Bwin_Winnings_Mar,Games_Bwin_Bets_Mar
1324454,1,7.85,2.950,8
1324615,3,44.50,31.500,72
1324883,2,11.00,3.250,22
1325165,1,31.00,25.850,12
1325187,8,649.02,602.147,741
1325381,1,2.00,3.000,2


In [66]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#April
#------------------------------------
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#March
UserDailyAggregation_raw_GB_Apr<-UserDailyAggregation_raw_GB[Date %between% c("2005-04-01","2005-04-30")]
#Frequency of Play
UserDailyAggregation_GB_agg_Apr<-UserDailyAggregation_raw_GB_Apr[,Games_Bwin_Freq_Apr:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_Apr<-UserDailyAggregation_raw_GB_Apr[,Games_Bwin_Stakes_Apr:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_Apr<-UserDailyAggregation_raw_GB_Apr[,Games_Bwin_Winnings_Apr:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_Apr<-UserDailyAggregation_raw_GB_Apr[,Games_Bwin_Bets_Apr:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_Apr<-unique(UserDailyAggregation_GB_agg_Apr,by="UserID")
UserDailyAggregation_GB_agg_Apr<-UserDailyAggregation_GB_agg_Apr[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_Apr)

UserID,Games_Bwin_Freq_Apr,Games_Bwin_Stakes_Apr,Games_Bwin_Winnings_Apr,Games_Bwin_Bets_Apr
1324379,1,122.17,126.078,51
1325636,6,9875.13,9329.415,1037
1325664,1,4.50,0.500,9
1326479,9,14.50,10.250,29
1326900,1,2.50,1.750,5
1327141,1,102.00,94.350,24


In [67]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#May
#------------------------------------
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#March
UserDailyAggregation_raw_GB_May<-UserDailyAggregation_raw_GB[Date %between% c("2005-05-01","2005-05-31")]
#Frequency of Play
UserDailyAggregation_GB_agg_May<-UserDailyAggregation_raw_GB_May[,Games_Bwin_Freq_May:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_May<-UserDailyAggregation_raw_GB_May[,Games_Bwin_Stakes_May:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_May<-UserDailyAggregation_raw_GB_May[,Games_Bwin_Winnings_May:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_May<-UserDailyAggregation_raw_GB_May[,Games_Bwin_Bets_May:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_May<-unique(UserDailyAggregation_GB_agg_May,by="UserID")
UserDailyAggregation_GB_agg_May<-UserDailyAggregation_GB_agg_May[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_May)

UserID,Games_Bwin_Freq_May,Games_Bwin_Stakes_May,Games_Bwin_Winnings_May,Games_Bwin_Bets_May
1324379,1,76.15,72.1500,42
1324644,1,3.00,3.9000,3
1324883,1,12.50,7.5000,25
1325636,1,784.04,667.7385,86
1326237,1,2.00,3.9000,2
1327567,1,26.45,5.4500,42


In [68]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#June
#------------------------------------
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#March
UserDailyAggregation_raw_GB_Jun<-UserDailyAggregation_raw_GB[Date %between% c("2005-06-01","2005-06-30")]
#Frequency of Play
UserDailyAggregation_GB_agg_Jun<-UserDailyAggregation_raw_GB_Jun[,Games_Bwin_Freq_Jun:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_Jun<-UserDailyAggregation_raw_GB_Jun[,Games_Bwin_Stakes_Jun:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_Jun<-UserDailyAggregation_raw_GB_Jun[,Games_Bwin_Winnings_Jun:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_Jun<-UserDailyAggregation_raw_GB_Jun[,Games_Bwin_Bets_Jun:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_Jun<-unique(UserDailyAggregation_GB_agg_Jun,by="UserID")
UserDailyAggregation_GB_agg_Jun<-UserDailyAggregation_GB_agg_Jun[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_Jun)

UserID,Games_Bwin_Freq_Jun,Games_Bwin_Stakes_Jun,Games_Bwin_Winnings_Jun,Games_Bwin_Bets_Jun
1324379,4,484.81,449.046,141
1324662,1,1.00,0.000,1
1324803,8,2867.47,2662.657,1869
1324935,1,30.50,29.950,26
1325187,2,26.95,20.400,29
1325636,9,14903.56,14918.096,1209


In [69]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#July
#------------------------------------
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#March
UserDailyAggregation_raw_GB_Jul<-UserDailyAggregation_raw_GB[Date %between% c("2005-07-01","2005-07-31")]
#Frequency of Play
UserDailyAggregation_GB_agg_Jul<-UserDailyAggregation_raw_GB_Jul[,Games_Bwin_Freq_Jul:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_Jul<-UserDailyAggregation_raw_GB_Jul[,Games_Bwin_Stakes_Jul:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_Jul<-UserDailyAggregation_raw_GB_Jul[,Games_Bwin_Winnings_Jul:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_Jul<-UserDailyAggregation_raw_GB_Jul[,Games_Bwin_Bets_Jul:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_Jul<-unique(UserDailyAggregation_GB_agg_Jul,by="UserID")
UserDailyAggregation_GB_agg_Jul<-UserDailyAggregation_GB_agg_Jul[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_Jul)

UserID,Games_Bwin_Freq_Jul,Games_Bwin_Stakes_Jul,Games_Bwin_Winnings_Jul,Games_Bwin_Bets_Jul
1324803,1,71.92,71.757,41
1325126,1,1.00,0.000,2
1325187,3,184.10,146.445,36
1325636,5,10195.05,9961.442,770
1326511,1,36.50,22.500,8
1326921,3,30.34,41.550,35


In [70]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#August
#------------------------------------
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#March
UserDailyAggregation_raw_GB_Aug<-UserDailyAggregation_raw_GB[Date %between% c("2005-08-01","2005-08-31")]
#Frequency of Play
UserDailyAggregation_GB_agg_Aug<-UserDailyAggregation_raw_GB_Aug[,Games_Bwin_Freq_Aug:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_Aug<-UserDailyAggregation_raw_GB_Aug[,Games_Bwin_Stakes_Aug:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_Aug<-UserDailyAggregation_raw_GB_Aug[,Games_Bwin_Winnings_Aug:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_Aug<-UserDailyAggregation_raw_GB_Aug[,Games_Bwin_Bets_Aug:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_Aug<-unique(UserDailyAggregation_GB_agg_Aug,by="UserID")
UserDailyAggregation_GB_agg_Aug<-UserDailyAggregation_GB_agg_Aug[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_Aug)

UserID,Games_Bwin_Freq_Aug,Games_Bwin_Stakes_Aug,Games_Bwin_Winnings_Aug,Games_Bwin_Bets_Aug
1324379,6,326.50,291.0750,191
1324803,1,38.81,28.8000,54
1324883,3,98.00,89.2500,187
1324935,1,13.50,9.0000,23
1325318,1,71.50,31.7500,45
1325636,1,596.53,514.0785,45


In [71]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#September
#------------------------------------
#Subset the User Daily Aggregation for Games Bwin to aggregate by user level
UserDailyAggregation_raw_GB<-UserDailyAggregation[ProductID==7,,]
#March
UserDailyAggregation_raw_GB_Sep<-UserDailyAggregation_raw_GB[Date %between% c("2005-09-01","2005-09-30")]
#Frequency of Play
UserDailyAggregation_GB_agg_Sep<-UserDailyAggregation_raw_GB_Sep[,Games_Bwin_Freq_Sep:=.N,by=UserID]
#total Stakes
UserDailyAggregation_GB_agg_Sep<-UserDailyAggregation_raw_GB_Sep[,Games_Bwin_Stakes_Sep:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_GB_agg_Sep<-UserDailyAggregation_raw_GB_Sep[,Games_Bwin_Winnings_Sep:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_GB_agg_Sep<-UserDailyAggregation_raw_GB_Sep[,Games_Bwin_Bets_Sep:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_GB_agg_Sep<-unique(UserDailyAggregation_GB_agg_Sep,by="UserID")
UserDailyAggregation_GB_agg_Sep<-UserDailyAggregation_GB_agg_Sep[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_GB_agg_Feb <- UserDailyAggregation_GB_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_GB_agg_Sep)

UserID,Games_Bwin_Freq_Sep,Games_Bwin_Stakes_Sep,Games_Bwin_Winnings_Sep,Games_Bwin_Bets_Sep
1325187,1,52.16,17.16,9
1325636,8,16063.05,15409.30,1265
1326919,1,7.00,5.25,14
1326921,2,1.50,3.00,3
1327141,3,142.00,134.35,53
1327258,2,42.00,33.75,77


In [72]:
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_Feb, by = "UserID", all.x = TRUE)
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_Mar, by = "UserID", all.x = TRUE)
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_Apr, by = "UserID", all.x = TRUE)
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_May, by = "UserID", all.x = TRUE)
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_Jun, by = "UserID", all.x = TRUE)
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_Jul, by = "UserID", all.x = TRUE)
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_Aug, by = "UserID", all.x = TRUE)
UserDailyAggregation_GB_agg<-merge(x = UserDailyAggregation_GB_agg, y = UserDailyAggregation_GB_agg_Sep, by = "UserID", all.x = TRUE)
#Droping the unnecessary columns

#cheking the aggregated table result
head(UserDailyAggregation_GB_agg)

UserID,Games_Bwin_Dur,Games_Bwin_1Day_Retention,Games_Bwin_Recent,Games_Bwin_Freq_Total,Games_Bwin_Stakes_Total,Games_Bwin_Winnings_Total,Games_Bwin_Bets_Total,Games_Bwin_Freq_FEB,Games_Bwin_Stakes_FEB,...,Games_Bwin_Winnings_Jul,Games_Bwin_Bets_Jul,Games_Bwin_Freq_Aug,Games_Bwin_Stakes_Aug,Games_Bwin_Winnings_Aug,Games_Bwin_Bets_Aug,Games_Bwin_Freq_Sep,Games_Bwin_Stakes_Sep,Games_Bwin_Winnings_Sep,Games_Bwin_Bets_Sep
1324369,11 days,Not_Rerturned,Not_Recent,2,2.00,0.000,2,2,2.00,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324379,133 days,Not_Rerturned,Not_Recent,12,1009.63,938.349,425,NA,NA,...,NA,NA,6,326.5,291.075,191,NA,NA,NA,NA
1324408,0 days,NA,Not_Recent,1,1.00,0.500,2,1,1.00,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324437,6 days,Returned,Not_Recent,6,866.00,716.000,215,6,866.00,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324454,28 days,Not_Rerturned,Not_Recent,4,28.10,8.200,32,3,20.25,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324615,21 days,Not_Rerturned,Not_Recent,3,44.50,31.500,72,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [73]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#First Day of Play
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Fisrt_Date:=Date[1],by=UserID]
#Last Day of Play
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Last_Date:=Date[.N],by=UserID]
#Duration between First and Last
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Dur:=difftime(Casino_Chart_Last_Date,
                                                            Casino_Chart_Fisrt_Date,units="days"),by=UserID]
#One Day Retention Calculation
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Second_Date:=Date[2],by=UserID]
#If customet return after 1 day or no
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,
                            Casino_Chart_1Day_Retention:=.(ifelse(Casino_Chart_Second_Date==Casino_Chart_Fisrt_Date+days(1),"Returned","Not_Rerturned")),]
#If customer play the game in last week or not
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,
                            Casino_Chart_Recent:=.(ifelse(Casino_Chart_Last_Date>="2005-09-23","Recent","Not_Recent")),]
#Frequency of Play
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Freq_Total:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Stakes_Total:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Winnings_Total:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg<-UserDailyAggregation_raw_CC[,Casino_Chart_Bets_Total:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg<-unique(UserDailyAggregation_CC_agg,by="UserID")
#Droping the unnecessary columns
UserDailyAggregation_CC_agg<-UserDailyAggregation_CC_agg[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL,Casino_Chart_Second_Date=NULL,
                                Casino_Chart_Fisrt_Date=NULL,Casino_Chart_Last_Date=NULL),]

head(UserDailyAggregation_CC_agg)

UserID,Casino_Chart_Dur,Casino_Chart_1Day_Retention,Casino_Chart_Recent,Casino_Chart_Freq_Total,Casino_Chart_Stakes_Total,Casino_Chart_Winnings_Total,Casino_Chart_Bets_Total
1324360,0 days,NA,Not_Recent,1,4.00,2.00,4
1324363,18 days,Not_Rerturned,Not_Recent,3,234.00,181.50,139
1324368,3 days,Not_Rerturned,Not_Recent,2,463.50,425.50,327
1324369,63 days,Not_Rerturned,Not_Recent,5,760.00,742.00,603
1324383,7 days,Not_Rerturned,Not_Recent,4,5529.53,4892.50,168
1324386,183 days,Not_Rerturned,Not_Recent,10,373.72,331.78,317


In [74]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#February
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#Calculatinf Frequency and total for February
UserDailyAggregation_raw_CC_Feb<-UserDailyAggregation_raw_CC[Date %between% c("2005-02-01","2005-02-28")]
#Frequency of Play
UserDailyAggregation_CC_agg_Feb<-UserDailyAggregation_raw_CC_Feb[,Casino_Chart_Freq_FEB:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_Feb<-UserDailyAggregation_raw_CC_Feb[,Casino_Chart_Stakes_FEB:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_Feb<-UserDailyAggregation_raw_CC_Feb[,Casino_Chart_Winnings_FEB:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_Feb<-UserDailyAggregation_raw_CC_Feb[,Casino_Chart_Bets_FEB:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_Feb<-unique(UserDailyAggregation_CC_agg_Feb,by="UserID")

UserDailyAggregation_CC_agg_Feb<-UserDailyAggregation_CC_agg_Feb[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_Feb)

UserID,Casino_Chart_Freq_FEB,Casino_Chart_Stakes_FEB,Casino_Chart_Winnings_FEB,Casino_Chart_Bets_FEB
1324360,1,4.00,2.00,4
1324363,3,234.00,181.50,139
1324386,1,158.00,156.00,152
1324400,1,1.50,0.00,2
1324408,4,132.13,99.96,96
1324432,1,1.00,0.00,1


In [75]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#MARCH
#------------------------------------
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#March
UserDailyAggregation_raw_CC_Mar<-UserDailyAggregation_raw_CC[Date %between% c("2005-03-01","2005-03-31")]
#Frequency of Play
UserDailyAggregation_CC_agg_Mar<-UserDailyAggregation_raw_CC_Mar[,Casino_Chart_Freq_Mar:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_Mar<-UserDailyAggregation_raw_CC_Mar[,Casino_Chart_Stakes_Mar:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_Mar<-UserDailyAggregation_raw_CC_Mar[,Casino_Chart_Winnings_Mar:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_Mar<-UserDailyAggregation_raw_CC_Mar[,Casino_Chart_Bets_Mar:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_Mar<-unique(UserDailyAggregation_CC_agg_Mar,by="UserID")
UserDailyAggregation_CC_agg_Mar<-UserDailyAggregation_CC_agg_Mar[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_Mar)

UserID,Casino_Chart_Freq_Mar,Casino_Chart_Stakes_Mar,Casino_Chart_Winnings_Mar,Casino_Chart_Bets_Mar
1324369,2,62.00,56.50,62
1324400,3,209.76,174.32,1337
1324408,8,2774.50,2500.34,1037
1324503,14,5050.50,4635.00,472
1324591,2,13.00,8.50,13
1324615,2,1095.25,963.75,380


In [76]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#April
#------------------------------------
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#March
UserDailyAggregation_raw_CC_Apr<-UserDailyAggregation_raw_CC[Date %between% c("2005-04-01","2005-04-30")]
#Frequency of Play
UserDailyAggregation_CC_agg_Apr<-UserDailyAggregation_raw_CC_Apr[,Casino_Chart_Freq_Apr:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_Apr<-UserDailyAggregation_raw_CC_Apr[,Casino_Chart_Stakes_Apr:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_Apr<-UserDailyAggregation_raw_CC_Apr[,Casino_Chart_Winnings_Apr:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_Apr<-UserDailyAggregation_raw_CC_Apr[,Casino_Chart_Bets_Apr:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_Apr<-unique(UserDailyAggregation_CC_agg_Apr,by="UserID")
UserDailyAggregation_CC_agg_Apr<-UserDailyAggregation_CC_agg_Apr[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_Apr)

UserID,Casino_Chart_Freq_Apr,Casino_Chart_Stakes_Apr,Casino_Chart_Winnings_Apr,Casino_Chart_Bets_Apr
1324369,2,695.00,684.50,538
1324386,3,118.00,103.50,103
1324408,7,2642.33,2511.33,609
1324495,2,165.00,112.50,11
1324503,5,4157.00,4447.00,125
1324749,7,2082.50,2062.50,282


In [77]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#May
#------------------------------------
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#March
UserDailyAggregation_raw_CC_May<-UserDailyAggregation_raw_CC[Date %between% c("2005-05-01","2005-05-31")]
#Frequency of Play
UserDailyAggregation_CC_agg_May<-UserDailyAggregation_raw_CC_May[,Casino_Chart_Freq_May:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_May<-UserDailyAggregation_raw_CC_May[,Casino_Chart_Stakes_May:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_May<-UserDailyAggregation_raw_CC_May[,Casino_Chart_Winnings_May:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_May<-UserDailyAggregation_raw_CC_May[,Casino_Chart_Bets_May:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_May<-unique(UserDailyAggregation_CC_agg_May,by="UserID")
UserDailyAggregation_CC_agg_May<-UserDailyAggregation_CC_agg_May[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_May)

UserID,Casino_Chart_Freq_May,Casino_Chart_Stakes_May,Casino_Chart_Winnings_May,Casino_Chart_Bets_May
1324369,1,3.0,1,3
1324386,1,11.0,9,6
1324408,2,55.5,51,40
1324503,4,751.0,550,43
1324591,2,41.0,38,41
1324749,2,100.0,140,13


In [78]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#June
#------------------------------------
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#March
UserDailyAggregation_raw_CC_Jun<-UserDailyAggregation_raw_CC[Date %between% c("2005-06-01","2005-06-30")]
#Frequency of Play
UserDailyAggregation_CC_agg_Jun<-UserDailyAggregation_raw_CC_Jun[,Casino_Chart_Freq_Jun:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_Jun<-UserDailyAggregation_raw_CC_Jun[,Casino_Chart_Stakes_Jun:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_Jun<-UserDailyAggregation_raw_CC_Jun[,Casino_Chart_Winnings_Jun:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_Jun<-UserDailyAggregation_raw_CC_Jun[,Casino_Chart_Bets_Jun:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_Jun<-unique(UserDailyAggregation_CC_agg_Jun,by="UserID")
UserDailyAggregation_CC_agg_Jun<-UserDailyAggregation_CC_agg_Jun[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_Jun)

UserID,Casino_Chart_Freq_Jun,Casino_Chart_Stakes_Jun,Casino_Chart_Winnings_Jun,Casino_Chart_Bets_Jun
1324383,4,5529.53,4892.50,168
1324466,4,150.50,133.00,136
1324503,1,1254.00,1168.00,49
1324808,1,0.00,0.00,0
1324935,8,3586.50,3649.50,1141
1325027,3,689.50,643.05,251


In [79]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#July
#------------------------------------
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#March
UserDailyAggregation_raw_CC_Jul<-UserDailyAggregation_raw_CC[Date %between% c("2005-07-01","2005-07-31")]
#Frequency of Play
UserDailyAggregation_CC_agg_Jul<-UserDailyAggregation_raw_CC_Jul[,Casino_Chart_Freq_Jul:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_Jul<-UserDailyAggregation_raw_CC_Jul[,Casino_Chart_Stakes_Jul:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_Jul<-UserDailyAggregation_raw_CC_Jul[,Casino_Chart_Winnings_Jul:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_Jul<-UserDailyAggregation_raw_CC_Jul[,Casino_Chart_Bets_Jul:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_Jul<-unique(UserDailyAggregation_CC_agg_Jul,by="UserID")
UserDailyAggregation_CC_agg_Jul<-UserDailyAggregation_CC_agg_Jul[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_Jul)

UserID,Casino_Chart_Freq_Jul,Casino_Chart_Stakes_Jul,Casino_Chart_Winnings_Jul,Casino_Chart_Bets_Jul
1324386,2,53.72,41.78,37
1324405,1,1.40,1.20,14
1325161,1,4.00,2.00,4
1325165,2,1718.73,1843.05,274
1325237,3,142.00,115.00,34
1325313,2,1100.00,900.00,130


In [80]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#August
#------------------------------------
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#March
UserDailyAggregation_raw_CC_Aug<-UserDailyAggregation_raw_CC[Date %between% c("2005-08-01","2005-08-31")]
#Frequency of Play
UserDailyAggregation_CC_agg_Aug<-UserDailyAggregation_raw_CC_Aug[,Casino_Chart_Freq_Aug:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_Aug<-UserDailyAggregation_raw_CC_Aug[,Casino_Chart_Stakes_Aug:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_Aug<-UserDailyAggregation_raw_CC_Aug[,Casino_Chart_Winnings_Aug:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_Aug<-UserDailyAggregation_raw_CC_Aug[,Casino_Chart_Bets_Aug:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_Aug<-unique(UserDailyAggregation_CC_agg_Aug,by="UserID")
UserDailyAggregation_CC_agg_Aug<-UserDailyAggregation_CC_agg_Aug[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_Aug)

UserID,Casino_Chart_Freq_Aug,Casino_Chart_Stakes_Aug,Casino_Chart_Winnings_Aug,Casino_Chart_Bets_Aug
1324368,2,463.50,425.50,327
1324386,3,33.00,21.50,19
1324405,1,0.60,0.00,6
1324786,1,8.25,8.25,33
1324883,1,40.00,20.00,52
1324935,4,775.00,798.00,50


In [81]:
#Importing user daily csv that we saved
UserDailyAggregation_raw<-fread("./Data/UserDailyAggregation_raw.csv")
#converting the date column in UserDailyAggregation to date type
UserDailyAggregation_raw$Date<-ymd(UserDailyAggregation_raw$Date)
#merge the UserDailyAggregation with 
UserDailyAggregation<-merge(x = UserDailyAggregation_raw, y = ProductDescription, by = "ProductID", all.x = TRUE)

#September
#------------------------------------
#Subset the User Daily Aggregation for Casino Chart to aggregate by user level
UserDailyAggregation_raw_CC<-UserDailyAggregation[ProductID==8,,]
#March
UserDailyAggregation_raw_CC_Sep<-UserDailyAggregation_raw_CC[Date %between% c("2005-09-01","2005-09-30")]
#Frequency of Play
UserDailyAggregation_CC_agg_Sep<-UserDailyAggregation_raw_CC_Sep[,Casino_Chart_Freq_Sep:=.N,by=UserID]
#total Stakes
UserDailyAggregation_CC_agg_Sep<-UserDailyAggregation_raw_CC_Sep[,Casino_Chart_Stakes_Sep:=sum(Stakes),by=UserID]
#Total Winning
UserDailyAggregation_CC_agg_Sep<-UserDailyAggregation_raw_CC_Sep[,Casino_Chart_Winnings_Sep:=sum(Winnings),by=UserID]
#Total Bets
UserDailyAggregation_CC_agg_Sep<-UserDailyAggregation_raw_CC_Sep[,Casino_Chart_Bets_Sep:=sum(Bets),by=UserID]
#Droping the duplicate User Id 
UserDailyAggregation_CC_agg_Sep<-unique(UserDailyAggregation_CC_agg_Sep,by="UserID")
UserDailyAggregation_CC_agg_Sep<-UserDailyAggregation_CC_agg_Sep[,`:=`(Date=NULL,Stakes=NULL,Winnings=NULL,
                                Bets=NULL,ProductID=NULL,"Product Description"=NULL),]
UserDailyAggregation_CC_agg_Feb <- UserDailyAggregation_CC_agg_Feb %>% replace(is.na(.), 0)
head(UserDailyAggregation_CC_agg_Sep)

UserID,Casino_Chart_Freq_Sep,Casino_Chart_Stakes_Sep,Casino_Chart_Winnings_Sep,Casino_Chart_Bets_Sep
1324405,1,0.60,0.20,6
1324591,1,7.00,4.00,7
1324888,4,32.00,24.00,32
1324935,3,94.00,27.50,28
1325165,1,311.58,215.62,41
1325215,2,357.00,345.00,265


In [82]:
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_Feb, by = "UserID", all.x = TRUE)
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_Mar, by = "UserID", all.x = TRUE)
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_Apr, by = "UserID", all.x = TRUE)
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_May, by = "UserID", all.x = TRUE)
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_Jun, by = "UserID", all.x = TRUE)
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_Jul, by = "UserID", all.x = TRUE)
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_Aug, by = "UserID", all.x = TRUE)
UserDailyAggregation_CC_agg<-merge(x = UserDailyAggregation_CC_agg, y = UserDailyAggregation_CC_agg_Sep, by = "UserID", all.x = TRUE)
#Droping the unnecessary columns

#cheking the aggregated table result
head(UserDailyAggregation_CC_agg)

UserID,Casino_Chart_Dur,Casino_Chart_1Day_Retention,Casino_Chart_Recent,Casino_Chart_Freq_Total,Casino_Chart_Stakes_Total,Casino_Chart_Winnings_Total,Casino_Chart_Bets_Total,Casino_Chart_Freq_FEB,Casino_Chart_Stakes_FEB,...,Casino_Chart_Winnings_Jul,Casino_Chart_Bets_Jul,Casino_Chart_Freq_Aug,Casino_Chart_Stakes_Aug,Casino_Chart_Winnings_Aug,Casino_Chart_Bets_Aug,Casino_Chart_Freq_Sep,Casino_Chart_Stakes_Sep,Casino_Chart_Winnings_Sep,Casino_Chart_Bets_Sep
1324360,0 days,NA,Not_Recent,1,4.00,2.00,4,1,4,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324363,18 days,Not_Rerturned,Not_Recent,3,234.00,181.50,139,3,234,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324368,3 days,Not_Rerturned,Not_Recent,2,463.50,425.50,327,NA,NA,...,NA,NA,2,463.5,425.5,327,NA,NA,NA,NA
1324369,63 days,Not_Rerturned,Not_Recent,5,760.00,742.00,603,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324383,7 days,Not_Rerturned,Not_Recent,4,5529.53,4892.50,168,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324386,183 days,Not_Rerturned,Not_Recent,10,373.72,331.78,317,1,158,...,41.78,37,3,33.0,21.5,19,NA,NA,NA,NA


<h2 style="text-align: center;">Base table</h2>

In [83]:
#Creation of base table first stem merging Demographics table with Pay out Agregation
Base_table<-merge(x = Demographics, y = games_agg, by = "UserID", all.x = TRUE)
#
Base_table<-merge(x = Base_table, y = trans_agg, by = "UserID", all.x = TRUE)
#Creation of base table first stem merging Demographics table with Pay out Agregation
Base_table<-merge(x = Base_table, y = PokerChipConversions_Sell_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Pay in Aggregation
Base_table<-merge(x = Base_table, y = PokerChipConversions_Buy_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Aggregation
Base_table<-merge(x = Base_table, y = UserDailyAggregation_SBFO_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Aggregation
Base_table<-merge(x = Base_table, y = UserDailyAggregation_SBLA_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Aggregation
Base_table<-merge(x = Base_table, y = UserDailyAggregation_CBM_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Aggregation
Base_table<-merge(x = Base_table, y = UserDailyAggregation_Supertoto_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Aggregation
Base_table<-merge(x = Base_table, y = UserDailyAggregation_GVS_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Aggregation
Base_table<-merge(x = Base_table, y = UserDailyAggregation_GB_agg, by = "UserID", all.x = TRUE)
#Merging Base table with Aggregation
Base_table<-merge(x = Base_table, y = UserDailyAggregation_CC_agg, by = "UserID", all.x = TRUE)
#------------------------------------------------------------------
#base table as data frame
Base_table<-data.frame(Base_table)
#------------------------------------------------------------------
#there is so many Na variables after merging all tables
#we dont need to change the NAs for numerical variables but we change NAs of categorical variables to z
# so we can easily filter them when we want to create plots or have analysis 
#-------------------------------------------------
#recency and retention variables
#
Base_table<-Base_table%>%mutate(Recency=replace_na(Recency,FALSE))
Base_table<-Base_table%>%mutate(Total.Retantion=replace_na(Total.Retantion,FALSE))

Base_table<-Base_table%>%mutate(Fix_Odd_1Day_Retention=replace_na(Fix_Odd_1Day_Retention,"z"))
Base_table<-Base_table%>%mutate(Fix_Odd_Recent=replace_na(Fix_Odd_Recent,"z"))

Base_table<-Base_table%>%mutate(Live_Act_1Day_Retention=replace_na(Live_Act_1Day_Retention,"z"))
Base_table<-Base_table%>%mutate(Live_Act_Recent=replace_na(Live_Act_Recent,"z"))

Base_table<-Base_table%>%mutate(Casino_Boss_1Day_Retention=replace_na(Casino_Boss_1Day_Retention,"z"))
Base_table<-Base_table%>%mutate(Casino_Boss_Recent=replace_na(Casino_Boss_Recent,"z"))

Base_table<-Base_table%>%mutate(Supertoto_1Day_Retention=replace_na(Supertoto_1Day_Retention,"z"))
Base_table<-Base_table%>%mutate(Supertoto_Recent=replace_na(Supertoto_Recent,"z"))

Base_table<-Base_table%>%mutate(Games_VS_1Day_Retention=replace_na(Games_VS_1Day_Retention,"z"))
Base_table<-Base_table%>%mutate(Games_VS_Recent=replace_na(Games_VS_Recent,"z"))

Base_table<-Base_table%>%mutate(Games_Bwin_1Day_Retention=replace_na(Games_Bwin_1Day_Retention,"z"))
Base_table<-Base_table%>%mutate(Games_Bwin_Recent=replace_na(Games_Bwin_Recent,"z"))

Base_table<-Base_table%>%mutate(Casino_Chart_1Day_Retention=replace_na(Casino_Chart_1Day_Retention,"z"))
Base_table<-Base_table%>%mutate(Casino_Chart_Recent=replace_na(Casino_Chart_Recent,"z"))
#----------------------------------------------
# KPIs and customes marketing groups variables
Base_table<-Base_table%>%mutate(Engagement_level=replace_na(Engagement_level,"z"))
Base_table<-Base_table%>%mutate(Loyalty_level=replace_na(Loyalty_level,"z"))
Base_table<-Base_table%>%mutate(game_Bets_groups=replace_na(game_Bets_groups,"z"))
Base_table<-Base_table%>%mutate(game_wining_groups=replace_na(game_wining_groups,"z"))
Base_table<-Base_table%>%mutate(game_stak_groups=replace_na(game_stak_groups,"z"))
Base_table<-Base_table%>%mutate(Game_freq_Groups=replace_na(Game_freq_Groups,"z"))
Base_table<-Base_table%>%mutate(Recency_Groups=replace_na(Recency_Groups,"z"))
#--------------------------------------------------------
#Exporting the Basetable to csv file for using it for analysis part and creation of Data Mart
fwrite(Base_table,"./Data/Base_table_month.csv")
#Seeing the first 5 rows of table
head(Base_table)

UserID,RegDate,FirstPay,FirstAct,FirstSp,FirstCa,FirstGa,FirstPo,Gender,Country,...,Casino_Chart_Winnings_Jul,Casino_Chart_Bets_Jul,Casino_Chart_Freq_Aug,Casino_Chart_Stakes_Aug,Casino_Chart_Winnings_Aug,Casino_Chart_Bets_Aug,Casino_Chart_Freq_Sep,Casino_Chart_Stakes_Sep,Casino_Chart_Winnings_Sep,Casino_Chart_Bets_Sep
1324354,2005-02-01,2005-02-24,2005-02-24,2005-02-24,NA,NA,NA,Male,Germany,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324355,2005-02-01,2005-02-01,2005-02-01,2005-02-01,NA,NA,2005-06-11,Male,Greece,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324356,2005-02-01,2005-02-01,2005-02-02,2005-02-02,NA,NA,NA,Male,Germany,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324358,2005-02-01,2005-02-01,2005-02-01,2005-02-01,NA,NA,NA,Male,Sweden,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324360,2005-02-01,2005-02-02,2005-02-02,2005-02-02,2005-02-03,NA,NA,Male,Turkey,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1324362,2005-02-01,2005-02-11,2005-02-11,2005-02-11,NA,NA,NA,Male,Germany,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [85]:
str(Base_table)

'data.frame':	42647 obs. of  342 variables:
 $ UserID                     : int  1324354 1324355 1324356 1324358 1324360 1324362 1324363 1324364 1324368 1324369 ...
 $ RegDate                    : IDate, format: "2005-02-01" "2005-02-01" ...
 $ FirstPay                   : Date, format: "2005-02-24" "2005-02-01" ...
 $ FirstAct                   : Date, format: "2005-02-24" "2005-02-01" ...
 $ FirstSp                    : Date, format: "2005-02-24" "2005-02-01" ...
 $ FirstCa                    : Date, format: NA NA ...
 $ FirstGa                    : Date, format: NA NA ...
 $ FirstPo                    : Date, format: NA "2005-06-11" ...
 $ Gender                     : chr  "Male" "Male" "Male" "Male" ...
 $ Country                    : chr  "Germany" "Greece" "Germany" "Sweden" ...
 $ CODE                       : Factor w/ 221 levels "ABW","AFG","AGO",..: 52 80 52 188 202 52 55 52 165 202 ...
 $ Language                   : chr  "German" "Greek" "German" "English" ...
 $ Application